The main goal of this notebook is to guess the documents used in the test dataset of stage 2 (excluding machine-generated data) and to guess their classes.  
I try to find the combination of 24% of the test dataset that scores my private leaderboard score.  
I've used Genetic Algorithm with the deap library (`pip install deap`).  
/!\ Like every GA, it cannot guarantee that the solution found is the best one ! /!\  
/!\ It is possible that several solutions have the same (or very close) log loss /!\  
Hope it's clear, bugless, accurate and not useless. :-)  

In [ ]:
import math
from random import *
from io import StringIO

import pandas as pd
import numpy as np

from sklearn.metrics import log_loss

from deap import creator, base, tools, algorithms

In [ ]:
# THIS 2 CELLS ARE USED TO GENERATE CSV STRING USED LATER
sub_files = ['./output/submissionFile_1506975743', './output/submissionFile_1506953486'] # change to the paths of your submitted files
pd.read_csv(sub_files[0], index_col='ID').to_csv(float_format='%.5f')

In [ ]:
# THIS 2 CELLS ARE USED TO GENERATE CSV STRING USED LATER
pd.read_csv(sub_files[1], index_col='ID').to_csv(float_format='%.5f')

In [ ]:
creator.create("IndFitness", base.Fitness, weights=(1.0,))

class Ind:
    def __init__(self, helper, IDs=None, classes=None):
        # init fitness
        self.fitness = creator.IndFitness()
        if isinstance(IDs, np.ndarray) and isinstance(classes, np.ndarray):
            self.IDs = IDs
            self.classes  = classes
        else:
            self.IDs, self.classes = helper.pickRandomInd()
    
    # print Ind
    def prettyPrint(self):
        print('#######')
        print('size: {}'.format(self.IDs.shape[0]))
        print('documents used in test:')
        print(self.IDs)
        print('classes of these documents:')
        print(self.classes)
        print('#######')

class Helper:
    def __init__(self, df_subs, score, perc_to_pick, mut_count_max):
        self.df_subs = df_subs
        self.score = score
        self.nb_doc = self.df_subs[0].shape[0]
        self.nb_to_pick = [math.floor(self.nb_doc*perc_to_pick), math.ceil(self.nb_doc*perc_to_pick)]
        self.mut_count_max = mut_count_max
    
    def pickRandomInd(self):
        n = choice(self.nb_to_pick)
        d = np.random.choice(np.arange(1,self.nb_doc+1), n, replace=False)
        c = np.random.randint(1, 10, n)
        idx = np.argsort(d)
        return d[idx], c[idx]
    
    # evaluation func
    def evalInd(self, ind):
        best_score = -float("inf")
        for df_sub in self.df_subs:
            y_pred = df_sub[df_sub.index.isin(ind.IDs)]
            score = -abs(self.score - log_loss(ind.classes, y_pred, labels=[1,2,3,4,5,6,7,8,9]))
            if score > best_score:
                best_score = score
        return best_score,

    # cross over func
    def cxInd(self, ind1, ind2):
        i, c = self._cxInd(ind1, ind2)
        n_ind1 = Ind(self, i, c)
        i, c = self._cxInd(ind1, ind2)
        n_ind2 = Ind(self, i, c)
        return n_ind1, n_ind2
    
    def _cxInd(self, ind1, ind2):
        # concatenate
        IDs = np.append(ind1.IDs, ind2.IDs)
        classes = np.append(ind1.classes, ind2.classes)
        # unique
        _, idx_u = np.unique(IDs, return_index=True)
        IDs = IDs[idx_u]
        classes = classes[idx_u]
        # pick random
        idx = np.sort(np.random.choice(np.arange(0,IDs.shape[0]), min(ind1.IDs.shape[0], choice(self.nb_to_pick)), replace=False))
        IDs = IDs[idx]
        classes = classes[idx]
        return IDs, classes

    # mutation func
    def mutInd(self, ind):
        IDs = ind.IDs.copy()
        classes = ind.classes.copy()
        for m in range(randint(1, self.mut_count_max)):
            # pick index to mutate
            idx = randint(0, IDs.shape[0]-1)
            # choose mutate type and pick new ID
            if randint(0, 1) == 0:
                # keep same document but change class
                n_ID = IDs[idx]
            else:
                # change both document and class
                n_ID = np.random.choice([i for i in self.df_subs[0].index if i not in IDs])
            # pick new class
            n_class = randint(1, 9)
            # mutate
            IDs[idx] = n_ID
            classes[idx] = n_class
        # TODO : add a mutation in order to change the size of the Ind
        # sort
        idx_sort = np.argsort(IDs)
        IDs = IDs[idx_sort]
        classes = classes[idx_sort]
        n_ind = Ind(self, IDs, classes)
        return n_ind,

In [ ]:
df_subs = []
df_sub1 = pd.read_csv(StringIO('ID,class1,class2,class3,class4,class5,class6,class7,class8,class9\n1,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n2,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n3,0.00129,0.30383,0.00084,0.00889,0.00120,0.00057,0.65764,0.00432,0.02142\n4,0.51136,0.01241,0.00001,0.00031,0.00001,0.00008,0.45745,0.01667,0.00170\n5,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n6,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n7,0.00006,0.23009,0.00014,0.00932,0.00096,0.00203,0.75477,0.00262,0.00001\n8,0.00990,0.08684,0.00454,0.53110,0.01432,0.09560,0.25360,0.00383,0.00027\n9,0.75676,0.00667,0.00070,0.22345,0.00191,0.00300,0.00427,0.00272,0.00052\n10,0.00012,0.02432,0.17858,0.00015,0.07120,0.00007,0.72547,0.00009,0.00000\n11,0.00263,0.10553,0.00017,0.71471,0.00770,0.01141,0.15760,0.00025,0.00000\n12,0.01192,0.81433,0.00757,0.01338,0.01149,0.00935,0.12318,0.00455,0.00423\n13,0.92368,0.00028,0.00011,0.01262,0.05290,0.00730,0.00060,0.00101,0.00150\n14,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n15,0.00712,0.40879,0.00136,0.02545,0.01285,0.01306,0.52775,0.00360,0.00002\n16,0.47762,0.17516,0.00401,0.11009,0.07644,0.09121,0.03757,0.02765,0.00025\n17,0.01638,0.08538,0.00018,0.00122,0.00037,0.00426,0.88934,0.00285,0.00002\n18,0.02970,0.71589,0.00104,0.01386,0.00582,0.01334,0.15627,0.05717,0.00691\n19,0.01866,0.07628,0.00574,0.02616,0.07020,0.00072,0.79870,0.00326,0.00028\n20,0.02768,0.46824,0.00025,0.00502,0.00405,0.02060,0.46359,0.00687,0.00370\n21,0.00042,0.08401,0.00031,0.00071,0.00128,0.00034,0.91186,0.00100,0.00007\n22,0.04671,0.00444,0.00000,0.14726,0.00001,0.00000,0.00281,0.01411,0.78466\n23,0.31180,0.01266,0.00000,0.00009,0.00002,0.00001,0.67540,0.00002,0.00000\n24,0.00005,0.59990,0.00001,0.00031,0.00076,0.00007,0.38963,0.00925,0.00002\n25,0.60714,0.00285,0.00126,0.01766,0.00632,0.00271,0.36091,0.00081,0.00034\n26,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n27,0.00008,0.85270,0.00000,0.04598,0.00001,0.01712,0.08405,0.00006,0.00000\n28,0.03589,0.18300,0.00214,0.72113,0.00173,0.00423,0.04588,0.00535,0.00065\n29,0.03019,0.18364,0.00030,0.76393,0.00013,0.00150,0.01937,0.00091,0.00003\n30,0.00001,0.87695,0.00080,0.00009,0.00007,0.03922,0.08285,0.00001,0.00000\n31,0.19935,0.05290,0.00527,0.01633,0.03001,0.08964,0.60630,0.00020,0.00000\n32,0.48631,0.01134,0.00003,0.50125,0.00000,0.00001,0.00099,0.00002,0.00005\n33,0.46234,0.01826,0.00047,0.51237,0.00141,0.00048,0.00445,0.00020,0.00002\n34,0.05388,0.11724,0.01436,0.05913,0.02594,0.03208,0.67809,0.01094,0.00834\n35,0.38224,0.45890,0.00001,0.03083,0.00002,0.00001,0.06575,0.00104,0.06120\n36,0.64040,0.00243,0.00075,0.34676,0.00422,0.00206,0.00269,0.00046,0.00023\n37,0.74590,0.02584,0.00012,0.07887,0.00396,0.14205,0.00301,0.00024,0.00001\n38,0.09586,0.46772,0.00575,0.04049,0.00921,0.02704,0.31954,0.01592,0.01848\n39,0.00053,0.05236,0.00002,0.00223,0.00010,0.00003,0.94459,0.00014,0.00000\n40,0.00606,0.09358,0.00017,0.00021,0.00054,0.00009,0.89916,0.00019,0.00000\n41,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n42,0.01527,0.82243,0.00023,0.00050,0.00487,0.00013,0.15529,0.00126,0.00002\n43,0.48757,0.35279,0.00001,0.04460,0.00051,0.00027,0.03276,0.08068,0.00081\n44,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n45,0.12150,0.00300,0.00011,0.87427,0.00056,0.00021,0.00026,0.00004,0.00005\n46,0.00048,0.03234,0.00008,0.00017,0.00030,0.64110,0.32514,0.00039,0.00000\n47,0.04360,0.12955,0.01245,0.04152,0.04367,0.02855,0.66756,0.02163,0.01147\n48,0.22959,0.00126,0.00114,0.74614,0.00017,0.00029,0.02140,0.00000,0.00001\n49,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n50,0.00688,0.34462,0.00506,0.00696,0.02219,0.00720,0.59337,0.01279,0.00093\n51,0.01883,0.04397,0.00011,0.46128,0.00143,0.00867,0.46459,0.00103,0.00009\n52,0.55014,0.00015,0.00193,0.05907,0.38416,0.00426,0.00027,0.00000,0.00002\n53,0.00141,0.20030,0.00005,0.31819,0.00009,0.00008,0.47943,0.00045,0.00000\n54,0.01500,0.20957,0.00293,0.02028,0.01385,0.00803,0.72467,0.00537,0.00030\n55,0.16773,0.01819,0.00001,0.00011,0.00002,0.00001,0.81078,0.00304,0.00011\n56,0.00044,0.05104,0.00041,0.00062,0.00142,0.00025,0.94485,0.00091,0.00006\n57,0.00574,0.00027,0.04581,0.93583,0.00663,0.00510,0.00042,0.00006,0.00014\n58,0.01200,0.16143,0.00158,0.81771,0.00062,0.00042,0.00570,0.00043,0.00011\n59,0.00724,0.47233,0.05849,0.42549,0.00083,0.00168,0.03325,0.00068,0.00000\n60,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n61,0.01626,0.49390,0.00009,0.00648,0.00274,0.34446,0.13168,0.00439,0.00001\n62,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n63,0.48433,0.02117,0.00001,0.01404,0.00007,0.00021,0.47928,0.00084,0.00004\n64,0.53535,0.00092,0.00267,0.44702,0.00901,0.00288,0.00180,0.00019,0.00016\n65,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n66,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n67,0.03217,0.49280,0.00682,0.02873,0.02575,0.02214,0.38221,0.00881,0.00058\n68,0.26574,0.24199,0.00000,0.01898,0.00023,0.00024,0.03137,0.43883,0.00261\n69,0.05990,0.15390,0.00210,0.26345,0.00465,0.00145,0.51205,0.00234,0.00016\n70,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n71,0.47580,0.00407,0.00001,0.00027,0.00001,0.00004,0.51977,0.00002,0.00001\n72,0.90722,0.05335,0.00001,0.00738,0.00179,0.00135,0.02156,0.00633,0.00101\n73,0.19456,0.40607,0.00370,0.01810,0.01940,0.01817,0.30146,0.03603,0.00250\n74,0.00054,0.11734,0.00073,0.00111,0.00265,0.00047,0.87526,0.00178,0.00012\n75,0.46046,0.00499,0.44508,0.06077,0.01185,0.00030,0.01654,0.00001,0.00000\n76,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n77,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n78,0.26027,0.00380,0.03844,0.68320,0.00869,0.00275,0.00263,0.00018,0.00004\n79,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n80,0.02012,0.01120,0.00563,0.11958,0.09305,0.72862,0.02134,0.00045,0.00001\n81,0.01122,0.55296,0.00131,0.34825,0.00102,0.00087,0.08269,0.00145,0.00023\n82,0.08819,0.01646,0.04771,0.11205,0.68714,0.01855,0.01786,0.00591,0.00613\n83,0.03609,0.01402,0.00067,0.24296,0.66523,0.02182,0.01909,0.00012,0.00000\n84,0.01306,0.03414,0.00607,0.02081,0.00983,0.00878,0.89745,0.00478,0.00508\n85,0.51813,0.12879,0.00057,0.00640,0.00569,0.00069,0.33241,0.00719,0.00013\n86,0.32967,0.58251,0.00038,0.02367,0.00209,0.00254,0.02701,0.02123,0.01090\n87,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n88,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n89,0.00981,0.02708,0.00129,0.00266,0.01797,0.00005,0.94067,0.00045,0.00002\n90,0.34613,0.61104,0.00001,0.03118,0.00004,0.00826,0.00291,0.00042,0.00001\n91,0.53185,0.00019,0.00003,0.01350,0.00248,0.45183,0.00011,0.00000,0.00001\n92,0.43872,0.01245,0.00882,0.03596,0.44518,0.03135,0.01405,0.00687,0.00659\n93,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n94,0.00072,0.51413,0.00003,0.00010,0.00008,0.00005,0.48471,0.00018,0.00000\n95,0.00163,0.17907,0.00146,0.00248,0.00438,0.00106,0.80780,0.00200,0.00012\n96,0.54416,0.00749,0.00011,0.01871,0.42787,0.00056,0.00104,0.00005,0.00001\n97,0.02838,0.74716,0.00014,0.22006,0.00003,0.00038,0.00385,0.00000,0.00000\n98,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n99,0.00003,0.06638,0.24142,0.00046,0.01001,0.00415,0.67751,0.00004,0.00000\n100,0.00006,0.37734,0.00001,0.00021,0.00022,0.00004,0.62040,0.00171,0.00001\n101,0.99957,0.00010,0.00000,0.00012,0.00002,0.00004,0.00013,0.00001,0.00001\n102,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n103,0.57924,0.05614,0.01761,0.20849,0.02024,0.02968,0.06151,0.01273,0.01436\n104,0.48277,0.37235,0.00001,0.04323,0.00044,0.00022,0.02961,0.07085,0.00053\n105,0.58267,0.01845,0.01058,0.30942,0.01682,0.01309,0.03258,0.00759,0.00880\n106,0.00364,0.33234,0.00640,0.01683,0.02440,0.01272,0.59591,0.00736,0.00040\n107,0.02651,0.01721,0.05145,0.01265,0.65764,0.03466,0.19965,0.00021,0.00002\n108,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n109,0.56157,0.02754,0.00096,0.04015,0.00305,0.00174,0.36439,0.00049,0.00011\n110,0.04007,0.02556,0.01363,0.08973,0.44302,0.11105,0.25653,0.01072,0.00969\n111,0.18452,0.06097,0.00010,0.04831,0.00116,0.04588,0.65842,0.00063,0.00001\n112,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n113,0.97515,0.01689,0.00000,0.00669,0.00001,0.00016,0.00057,0.00051,0.00002\n114,0.94537,0.00403,0.00001,0.04464,0.00003,0.00002,0.00374,0.00199,0.00017\n115,0.00998,0.02381,0.00458,0.01117,0.00762,0.00901,0.92721,0.00351,0.00311\n116,0.66657,0.00710,0.00211,0.29573,0.00895,0.00768,0.01077,0.00090,0.00019\n117,0.00004,0.27447,0.00192,0.00075,0.00136,0.00934,0.71161,0.00051,0.00000\n118,0.75053,0.00067,0.00005,0.24848,0.00007,0.00005,0.00014,0.00000,0.00001\n119,0.00033,0.00671,0.00000,0.00002,0.00001,0.00000,0.90787,0.00471,0.08035\n120,0.26408,0.07664,0.00017,0.04966,0.00342,0.00206,0.59684,0.00576,0.00137\n121,0.01999,0.01166,0.00791,0.00973,0.00153,0.93029,0.01729,0.00160,0.00000\n122,0.00004,0.20400,0.00004,0.00037,0.00093,0.00016,0.79411,0.00035,0.00000\n123,0.00002,0.05817,0.33902,0.00035,0.00667,0.00874,0.58701,0.00002,0.00000\n124,0.06359,0.14900,0.01678,0.12233,0.02675,0.04031,0.55528,0.01324,0.01272\n125,0.01000,0.40509,0.00021,0.16679,0.01873,0.26266,0.12589,0.01062,0.00000\n126,0.00159,0.85580,0.00033,0.01138,0.01187,0.03987,0.07824,0.00092,0.00000\n127,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n128,0.91978,0.00634,0.00006,0.04241,0.00073,0.02274,0.00651,0.00121,0.00022\n129,0.56265,0.10158,0.01275,0.12208,0.02968,0.02423,0.08906,0.02461,0.03336\n130,0.00018,0.00801,0.00002,0.00072,0.00034,0.97976,0.01009,0.00088,0.00000\n131,0.01777,0.06318,0.01031,0.05330,0.01417,0.01146,0.81638,0.00741,0.00602\n132,0.53058,0.33658,0.00001,0.03310,0.00028,0.00019,0.02861,0.07003,0.00062\n133,0.00004,0.22210,0.00016,0.00028,0.00118,0.00116,0.77258,0.00250,0.00000\n134,0.39948,0.06289,0.01728,0.29425,0.03995,0.03405,0.11047,0.02083,0.02080\n135,0.50756,0.03274,0.00006,0.00218,0.00021,0.01730,0.43985,0.00009,0.00001\n136,0.54668,0.27920,0.00003,0.04320,0.00123,0.08425,0.04353,0.00188,0.00000\n137,0.06550,0.74689,0.00110,0.00463,0.00589,0.00297,0.15395,0.01796,0.00111\n138,0.00027,0.62602,0.00013,0.00118,0.01014,0.03039,0.31061,0.02115,0.00011\n139,0.00763,0.00005,0.05868,0.93134,0.00100,0.00089,0.00041,0.00000,0.00000\n140,0.00177,0.00532,0.00002,0.84565,0.00001,0.00001,0.14721,0.00001,0.00000\n141,0.00038,0.04392,0.02698,0.00021,0.06454,0.00010,0.86294,0.00092,0.00001\n142,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n143,0.00004,0.00022,0.00000,0.00001,0.00000,0.00000,0.99973,0.00000,0.00000\n144,0.06008,0.08297,0.00185,0.85205,0.00061,0.00049,0.00162,0.00020,0.00013\n145,0.12683,0.04206,0.01538,0.65931,0.03152,0.01862,0.08293,0.01007,0.01328\n146,0.01466,0.06674,0.00852,0.01570,0.01237,0.01084,0.85101,0.01294,0.00722\n147,0.07064,0.00008,0.00004,0.92914,0.00002,0.00001,0.00007,0.00000,0.00000\n148,0.05812,0.02360,0.00007,0.91360,0.00001,0.00001,0.00367,0.00083,0.00009\n149,0.00150,0.16144,0.00072,0.00053,0.60906,0.00226,0.22403,0.00046,0.00000\n150,0.51781,0.01468,0.00236,0.44869,0.00361,0.00390,0.00524,0.00325,0.00046\n151,0.03265,0.11458,0.00080,0.01577,0.76324,0.00204,0.07036,0.00056,0.00000\n152,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n153,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n154,0.19307,0.00026,0.00009,0.80629,0.00007,0.00002,0.00020,0.00000,0.00000\n155,0.00407,0.98056,0.00001,0.00433,0.00003,0.00803,0.00193,0.00103,0.00001\n156,0.01663,0.38467,0.00514,0.01015,0.02228,0.01378,0.53601,0.01070,0.00064\n157,0.00694,0.09566,0.00437,0.00577,0.02315,0.00330,0.85654,0.00406,0.00021\n158,0.00001,0.05020,0.00001,0.00004,0.00002,0.00004,0.94966,0.00002,0.00000\n159,0.00011,0.03991,0.00047,0.00084,0.00078,0.01042,0.94647,0.00095,0.00005\n160,0.79546,0.11556,0.00011,0.06858,0.00015,0.00023,0.01956,0.00026,0.00009\n161,0.48448,0.37672,0.00001,0.04388,0.00045,0.00024,0.03191,0.06180,0.00051\n162,0.02710,0.00383,0.00001,0.00056,0.00003,0.84221,0.12624,0.00002,0.00000\n163,0.06899,0.01998,0.00391,0.70751,0.00502,0.00295,0.00604,0.03437,0.15123\n164,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n165,0.47066,0.37654,0.00001,0.04286,0.00054,0.00059,0.02796,0.08036,0.00049\n166,0.23367,0.72949,0.00000,0.01444,0.00001,0.00203,0.02034,0.00001,0.00001\n167,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n168,0.06955,0.20498,0.01439,0.08209,0.02169,0.02375,0.55927,0.01351,0.01077\n169,0.86294,0.00010,0.00002,0.13651,0.00011,0.00004,0.00004,0.00000,0.00024\n170,0.02465,0.04875,0.00125,0.31631,0.00050,0.08589,0.52182,0.00082,0.00001\n171,0.00026,0.65269,0.00001,0.00873,0.00041,0.00031,0.33686,0.00073,0.00000\n172,0.00541,0.02086,0.00008,0.96606,0.00004,0.00004,0.00747,0.00004,0.00000\n173,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n174,0.66049,0.02553,0.01459,0.16267,0.04101,0.03616,0.04148,0.00930,0.00877\n175,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n176,0.05361,0.04504,0.00288,0.00616,0.65249,0.00028,0.23766,0.00156,0.00032\n177,0.97518,0.00018,0.00000,0.01678,0.00001,0.00057,0.00013,0.00002,0.00713\n178,0.29884,0.00012,0.00309,0.51475,0.15157,0.03154,0.00008,0.00000,0.00001\n179,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n180,0.00663,0.36248,0.00098,0.01197,0.00285,0.00230,0.60411,0.00773,0.00095\n181,0.00008,0.52705,0.00000,0.00003,0.00003,0.00002,0.47271,0.00008,0.00000\n182,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n183,0.00141,0.31781,0.00427,0.00380,0.01299,0.00378,0.65189,0.00379,0.00026\n184,0.37709,0.00231,0.00174,0.04360,0.00556,0.56894,0.00069,0.00006,0.00001\n185,0.01374,0.01417,0.00267,0.62674,0.00155,0.00155,0.33874,0.00076,0.00008\n186,0.08422,0.00011,0.00005,0.91547,0.00004,0.00001,0.00010,0.00000,0.00000\n187,0.10947,0.17092,0.01310,0.03469,0.02930,0.02936,0.58587,0.01519,0.01210\n188,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n189,0.00005,0.37842,0.00001,0.00005,0.00023,0.00004,0.61734,0.00383,0.00003\n190,0.03211,0.19859,0.01299,0.04127,0.01726,0.01521,0.66768,0.00865,0.00624\n191,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n192,0.00004,0.29187,0.00013,0.00022,0.00148,0.00356,0.70066,0.00204,0.00000\n193,0.00539,0.03889,0.00077,0.72578,0.00110,0.00021,0.22782,0.00004,0.00000\n194,0.16905,0.00001,0.00880,0.16992,0.63972,0.01248,0.00002,0.00000,0.00000\n195,0.23861,0.24518,0.00504,0.02540,0.04363,0.00046,0.40241,0.03901,0.00027\n196,0.00339,0.00001,0.05466,0.94139,0.00021,0.00016,0.00018,0.00000,0.00000\n197,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n198,0.07338,0.22251,0.01272,0.06735,0.01885,0.55967,0.02845,0.00773,0.00934\n199,0.53461,0.02791,0.00000,0.00012,0.00000,0.00000,0.43722,0.00004,0.00010\n200,0.45036,0.23941,0.00013,0.00567,0.00326,0.21277,0.08458,0.00381,0.00000\n201,0.52895,0.33999,0.00001,0.03194,0.00036,0.00025,0.02557,0.07233,0.00060\n202,0.45074,0.00020,0.00021,0.00840,0.19877,0.34164,0.00004,0.00000,0.00000\n203,0.00476,0.52590,0.00052,0.00673,0.00199,0.00109,0.45541,0.00319,0.00041\n204,0.00001,0.28476,0.00000,0.00001,0.00002,0.00001,0.71396,0.00123,0.00000\n205,0.31730,0.00013,0.00532,0.04917,0.61522,0.01255,0.00026,0.00001,0.00004\n206,0.99951,0.00003,0.00000,0.00043,0.00000,0.00001,0.00001,0.00000,0.00001\n207,0.31286,0.07774,0.00733,0.10946,0.03577,0.07833,0.37699,0.00146,0.00006\n208,0.06828,0.25526,0.01635,0.07233,0.05329,0.02822,0.48605,0.01131,0.00890\n209,0.08516,0.00273,0.02250,0.07745,0.61123,0.19965,0.00099,0.00026,0.00003\n210,0.43726,0.01835,0.00047,0.00083,0.01188,0.00082,0.53015,0.00008,0.00016\n211,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n212,0.21847,0.00043,0.00041,0.77943,0.00075,0.00013,0.00038,0.00000,0.00000\n213,0.68923,0.02170,0.00000,0.02764,0.00001,0.00001,0.26096,0.00040,0.00005\n214,0.01792,0.61713,0.00042,0.05377,0.01664,0.27065,0.02175,0.00171,0.00001\n215,0.00052,0.12896,0.00063,0.00099,0.00258,0.00047,0.86389,0.00183,0.00013\n216,0.03986,0.03262,0.01106,0.03643,0.73022,0.01360,0.12050,0.00926,0.00645\n217,0.01192,0.81433,0.00757,0.01338,0.01149,0.00935,0.12318,0.00455,0.00423\n218,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n219,0.06012,0.00280,0.00542,0.09467,0.19148,0.00549,0.64001,0.00000,0.00000\n220,0.00027,0.03594,0.00703,0.00257,0.00148,0.34593,0.60666,0.00012,0.00000\n221,0.00000,0.03540,0.00001,0.00003,0.00002,0.00002,0.96450,0.00002,0.00000\n222,0.18692,0.77637,0.00000,0.00012,0.00093,0.00095,0.03412,0.00056,0.00003\n223,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n224,0.51939,0.34760,0.00001,0.03128,0.00038,0.00022,0.02827,0.07235,0.00050\n225,0.24867,0.68682,0.00002,0.01444,0.00083,0.00075,0.03342,0.01461,0.00044\n226,0.19508,0.01870,0.00018,0.54244,0.00063,0.00155,0.24137,0.00002,0.00003\n227,0.55397,0.02713,0.00003,0.01279,0.00004,0.00012,0.40589,0.00003,0.00000\n228,0.25937,0.18990,0.01100,0.02770,0.07778,0.00043,0.40299,0.03051,0.00032\n229,0.49798,0.35370,0.00003,0.04962,0.00039,0.00020,0.02681,0.07065,0.00063\n230,0.08236,0.00823,0.00015,0.36874,0.00075,0.00003,0.53907,0.00061,0.00006\n231,0.00092,0.00197,0.00005,0.46319,0.00001,0.00000,0.53385,0.00001,0.00000\n232,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n233,0.04288,0.00505,0.00002,0.29511,0.00003,0.00002,0.65684,0.00005,0.00000\n234,0.20481,0.00891,0.00112,0.00158,0.00629,0.00127,0.77463,0.00093,0.00046\n235,0.92985,0.00131,0.00021,0.00307,0.06223,0.00046,0.00280,0.00002,0.00005\n236,0.48878,0.20036,0.01411,0.07501,0.10099,0.01769,0.08485,0.00938,0.00883\n237,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n238,0.05761,0.24385,0.00889,0.04510,0.01541,0.01880,0.59228,0.00798,0.01008\n239,0.53661,0.00078,0.00119,0.45475,0.00337,0.00172,0.00128,0.00015,0.00015\n240,0.83017,0.00214,0.00003,0.16635,0.00020,0.00010,0.00084,0.00005,0.00012\n241,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n242,0.00009,0.99379,0.00000,0.00003,0.00004,0.00092,0.00492,0.00021,0.00000\n243,0.00326,0.00552,0.00048,0.66822,0.00483,0.00010,0.31756,0.00003,0.00000\n244,0.51510,0.03217,0.00016,0.45198,0.00003,0.00008,0.00021,0.00019,0.00008\n245,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n246,0.01616,0.47371,0.00001,0.50210,0.00001,0.00000,0.00799,0.00002,0.00000\n247,0.22816,0.16144,0.01794,0.15272,0.01245,0.04781,0.35839,0.02090,0.00020\n248,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n249,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n250,0.75966,0.01735,0.00003,0.07343,0.00036,0.00045,0.14867,0.00002,0.00003\n251,0.17504,0.41641,0.00019,0.02041,0.00222,0.01080,0.34397,0.03067,0.00029\n252,0.03618,0.11343,0.00693,0.09514,0.02306,0.12261,0.60256,0.00009,0.00000\n253,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n254,0.27980,0.00477,0.00002,0.08944,0.00016,0.00020,0.62552,0.00007,0.00002\n255,0.51168,0.14887,0.02116,0.13086,0.02056,0.03417,0.09059,0.02518,0.01693\n256,0.22643,0.02052,0.01101,0.12731,0.01887,0.55467,0.02650,0.00740,0.00729\n257,0.10207,0.55462,0.00104,0.21082,0.04535,0.00173,0.08437,0.00000,0.00000\n258,0.00775,0.55938,0.00039,0.00304,0.00576,0.01172,0.37713,0.01784,0.01699\n259,0.01187,0.00032,0.00067,0.97986,0.00041,0.00680,0.00007,0.00000,0.00000\n260,0.23254,0.54706,0.00019,0.19058,0.00090,0.00374,0.02492,0.00005,0.00002\n261,0.24068,0.02794,0.00034,0.71628,0.00069,0.00023,0.01326,0.00045,0.00013\n262,0.01839,0.03554,0.00006,0.58846,0.00006,0.00006,0.35624,0.00116,0.00003\n263,0.00006,0.85314,0.00658,0.00369,0.01639,0.02485,0.09498,0.00031,0.00000\n264,0.24267,0.03672,0.01266,0.04603,0.58444,0.03404,0.02420,0.00961,0.00963\n265,0.50868,0.01469,0.01106,0.03641,0.37903,0.01794,0.01733,0.00741,0.00745\n266,0.00006,0.27637,0.00010,0.00031,0.00050,0.00059,0.72053,0.00154,0.00000\n267,0.00049,0.04441,0.00071,0.00068,0.00293,0.00045,0.94928,0.00098,0.00007\n268,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n269,0.00401,0.39222,0.00009,0.00502,0.00025,0.00011,0.59795,0.00035,0.00000\n270,0.00078,0.31606,0.00049,0.00178,0.00564,0.27313,0.39444,0.00767,0.00000\n271,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n272,0.00051,0.02277,0.00096,0.00066,0.00147,0.00020,0.97298,0.00041,0.00004\n273,0.24834,0.00277,0.00028,0.23157,0.51349,0.00092,0.00255,0.00006,0.00002\n274,0.61928,0.35195,0.00000,0.00095,0.00002,0.00016,0.02184,0.00260,0.00320\n275,0.00477,0.00087,0.00005,0.41829,0.00000,0.00000,0.57602,0.00000,0.00000\n276,0.22839,0.01328,0.00003,0.61301,0.00007,0.00009,0.14507,0.00006,0.00000\n277,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n278,0.61984,0.12841,0.01306,0.09516,0.01809,0.02305,0.06739,0.02115,0.01384\n279,0.00018,0.01724,0.00000,0.00010,0.00004,0.00000,0.00945,0.05945,0.91354\n280,0.01466,0.57831,0.00001,0.00307,0.00005,0.00211,0.40158,0.00021,0.00000\n281,0.38325,0.06839,0.00364,0.17971,0.16641,0.16414,0.03191,0.00251,0.00005\n282,0.05089,0.01792,0.00027,0.00044,0.00423,0.00006,0.92615,0.00004,0.00000\n283,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n284,0.56777,0.00319,0.00001,0.04428,0.00000,0.00001,0.38474,0.00000,0.00000\n285,0.23089,0.10229,0.02216,0.36970,0.06319,0.06137,0.12004,0.01514,0.01521\n286,0.65524,0.01964,0.00365,0.28780,0.00743,0.01224,0.00836,0.00495,0.00069\n287,0.00867,0.16030,0.00176,0.00752,0.00742,0.00431,0.17156,0.03039,0.60807\n288,0.18489,0.00143,0.00007,0.81306,0.00006,0.00004,0.00040,0.00001,0.00004\n289,0.00205,0.18415,0.00298,0.00674,0.01034,0.00373,0.78630,0.00352,0.00019\n290,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n291,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000\n292,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n293,0.52675,0.00226,0.00000,0.00046,0.00006,0.06770,0.38448,0.01818,0.00011\n294,0.04374,0.10138,0.01175,0.10645,0.01805,0.01607,0.68716,0.00813,0.00727\n295,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n296,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n297,0.67393,0.00077,0.00006,0.32279,0.00151,0.00015,0.00074,0.00001,0.00004\n298,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n299,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n300,0.00062,0.18718,0.00000,0.00036,0.00181,0.00043,0.13184,0.64594,0.03182\n301,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n302,0.01363,0.00774,0.00008,0.00033,0.00039,0.51320,0.46457,0.00006,0.00000\n303,0.00013,0.06642,0.00061,0.00056,0.00034,0.05178,0.36777,0.00743,0.50496\n304,0.00004,0.00043,0.00007,0.00265,0.00011,0.51371,0.48299,0.00000,0.00000\n305,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n306,0.00038,0.59542,0.04222,0.03033,0.00690,0.00120,0.32052,0.00302,0.00001\n307,0.71348,0.02212,0.00908,0.13688,0.01647,0.01352,0.07358,0.00745,0.00743\n308,0.50049,0.00363,0.00012,0.01368,0.09741,0.38339,0.00111,0.00017,0.00000\n309,0.00798,0.17982,0.00016,0.00396,0.00542,0.76050,0.03903,0.00313,0.00000\n310,0.00015,0.74912,0.00000,0.00024,0.00001,0.00004,0.25040,0.00004,0.00000\n311,0.21170,0.02021,0.00840,0.69507,0.01696,0.01266,0.01871,0.00696,0.00933\n312,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n313,0.28886,0.06150,0.00000,0.00374,0.00001,0.00039,0.64544,0.00006,0.00000\n314,0.45171,0.00084,0.00107,0.54299,0.00099,0.00107,0.00106,0.00014,0.00013\n315,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n316,0.09532,0.02867,0.01172,0.79194,0.01417,0.01357,0.02912,0.00846,0.00703\n317,0.48253,0.00162,0.00067,0.09746,0.13312,0.28452,0.00006,0.00001,0.00000\n318,0.00007,0.11988,0.00015,0.00026,0.00278,0.00042,0.87545,0.00098,0.00001\n319,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n320,0.33595,0.00063,0.00004,0.61168,0.00000,0.00000,0.05169,0.00000,0.00001\n321,0.27671,0.12324,0.00009,0.01281,0.00072,0.00069,0.58509,0.00064,0.00001\n322,0.22604,0.65485,0.00070,0.00872,0.00463,0.02131,0.01981,0.04270,0.02124\n323,0.07330,0.04781,0.01082,0.07187,0.02120,0.02094,0.73531,0.00976,0.00899\n324,0.00192,0.06608,0.00186,0.00629,0.00326,0.00211,0.70136,0.00790,0.20922\n325,0.02441,0.58863,0.00010,0.38565,0.00001,0.00025,0.00092,0.00003,0.00000\n326,0.00054,0.38102,0.00027,0.00050,0.00218,0.00724,0.60284,0.00524,0.00017\n327,0.46008,0.23836,0.01272,0.05211,0.02835,0.02183,0.15336,0.02216,0.01103\n328,0.81711,0.00120,0.00230,0.14738,0.01764,0.01229,0.00161,0.00029,0.00018\n329,0.00004,0.87770,0.00000,0.00004,0.00004,0.00008,0.12161,0.00049,0.00000\n330,0.28056,0.00065,0.02085,0.02674,0.21208,0.45859,0.00051,0.00002,0.00001\n331,0.65501,0.01339,0.00066,0.07497,0.00365,0.03378,0.18246,0.01562,0.02046\n332,0.10323,0.02981,0.00153,0.79820,0.00138,0.06548,0.00037,0.00000,0.00000\n333,0.54125,0.00009,0.00209,0.07276,0.37977,0.00382,0.00019,0.00000,0.00003\n334,0.00726,0.64286,0.00000,0.00107,0.00044,0.01314,0.33462,0.00061,0.00000\n335,0.00359,0.04441,0.00005,0.45264,0.00762,0.46339,0.02393,0.00436,0.00001\n336,0.10019,0.10422,0.01273,0.02999,0.02824,0.04181,0.65783,0.01464,0.01035\n337,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n338,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n339,0.21877,0.00895,0.00001,0.00005,0.00001,0.00003,0.74299,0.00165,0.02754\n340,0.25396,0.25661,0.00119,0.22288,0.00194,0.00685,0.24150,0.01488,0.00019\n341,0.29701,0.68307,0.00000,0.01203,0.00004,0.00004,0.00746,0.00034,0.00001\n342,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n343,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n344,0.00256,0.04651,0.00000,0.00015,0.00006,0.00050,0.01933,0.92921,0.00168\n345,0.99948,0.00005,0.00000,0.00021,0.00000,0.00000,0.00002,0.00001,0.00023\n346,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n347,0.02527,0.00017,0.00511,0.13773,0.81547,0.01574,0.00050,0.00000,0.00001\n348,0.00001,0.08952,0.00010,0.00002,0.00111,0.00006,0.90745,0.00172,0.00001\n349,0.00025,0.13076,0.00001,0.00054,0.00055,0.00007,0.19984,0.12007,0.54791\n350,0.08971,0.45817,0.00003,0.30079,0.00001,0.00001,0.15126,0.00002,0.00000\n351,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n352,0.67021,0.00716,0.00013,0.31713,0.00008,0.00027,0.00492,0.00008,0.00002\n353,0.00063,0.06307,0.00002,0.93608,0.00000,0.00009,0.00011,0.00000,0.00000\n354,0.36668,0.00010,0.00001,0.63314,0.00000,0.00001,0.00004,0.00000,0.00002\n355,0.00173,0.01968,0.00062,0.03857,0.00203,0.00004,0.93730,0.00003,0.00000\n356,0.54389,0.06966,0.00012,0.00057,0.01358,0.01950,0.35249,0.00017,0.00002\n357,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n358,0.84526,0.01677,0.00002,0.00590,0.00337,0.11138,0.01696,0.00033,0.00001\n359,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n360,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n361,0.00003,0.09982,0.00066,0.00031,0.00052,0.02435,0.87395,0.00036,0.00000\n362,0.30435,0.01015,0.00046,0.03043,0.00435,0.00192,0.63656,0.01076,0.00102\n363,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n364,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000\n365,0.14752,0.41218,0.00776,0.03541,0.00581,0.01059,0.34871,0.03182,0.00020\n366,0.07468,0.61393,0.00003,0.30769,0.00003,0.00001,0.00354,0.00009,0.00000\n367,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n368,0.05452,0.01687,0.01601,0.81008,0.04500,0.02083,0.01576,0.01065,0.01028\n369,0.69178,0.19343,0.00006,0.09575,0.00034,0.00167,0.01229,0.00442,0.00026\n370,0.02031,0.26360,0.01177,0.04076,0.01762,0.01690,0.61395,0.00775,0.00734\n371,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n372,0.74286,0.00041,0.00083,0.14907,0.06020,0.04648,0.00013,0.00001,0.00001\n373,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n374,0.00089,0.51300,0.00000,0.00094,0.00009,0.00002,0.48477,0.00029,0.00000\n375,0.00031,0.03285,0.00003,0.00211,0.00020,0.57018,0.39419,0.00013,0.00000\n376,0.13595,0.00003,0.02640,0.34628,0.43263,0.05865,0.00005,0.00000,0.00000\n377,0.34626,0.00186,0.00054,0.03612,0.03972,0.00034,0.57507,0.00008,0.00001\n378,0.84300,0.02363,0.00019,0.00653,0.00301,0.07514,0.04569,0.00269,0.00012\n379,0.56777,0.00284,0.00000,0.00004,0.00000,0.00000,0.42935,0.00000,0.00000\n380,0.00006,0.21396,0.00018,0.00016,0.00197,0.00017,0.78220,0.00128,0.00002\n381,0.00947,0.02109,0.00003,0.39959,0.00005,0.00002,0.56934,0.00040,0.00001\n382,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n383,0.00004,0.26008,0.00076,0.00122,0.00073,0.00030,0.73650,0.00037,0.00000\n384,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n385,0.24358,0.00008,0.01111,0.33259,0.34944,0.06313,0.00006,0.00000,0.00001\n386,0.00193,0.25224,0.01372,0.04595,0.05323,0.03218,0.59618,0.00455,0.00002\n387,0.00449,0.01430,0.00002,0.29020,0.00015,0.00010,0.69045,0.00029,0.00000\n388,0.03281,0.08856,0.00062,0.01967,0.00203,0.26998,0.57918,0.00715,0.00000\n389,0.72995,0.00059,0.00001,0.20601,0.06192,0.00011,0.00140,0.00000,0.00001\n390,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n391,0.02171,0.40255,0.00590,0.01259,0.54883,0.00749,0.00062,0.00030,0.00001\n392,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n393,0.00020,0.25153,0.00265,0.00098,0.02605,0.00065,0.71608,0.00182,0.00004\n394,0.05084,0.17625,0.01608,0.05192,0.02369,0.02413,0.63649,0.01185,0.00875\n395,0.54257,0.05172,0.01500,0.23314,0.02981,0.02211,0.07951,0.01421,0.01193\n396,0.20261,0.46445,0.00024,0.02174,0.00511,0.00607,0.27020,0.02933,0.00025\n397,0.00163,0.07215,0.00470,0.00308,0.01621,0.00273,0.89745,0.00191,0.00014\n398,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n399,0.89248,0.00261,0.00012,0.00228,0.09793,0.00101,0.00345,0.00006,0.00006\n400,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n401,0.00218,0.37402,0.00047,0.00087,0.00804,0.00701,0.60529,0.00211,0.00001\n402,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n403,0.18379,0.04838,0.02435,0.50888,0.04486,0.02534,0.13578,0.01450,0.01412\n404,0.54172,0.00097,0.00237,0.44192,0.00816,0.00256,0.00197,0.00018,0.00015\n405,0.01033,0.12256,0.00011,0.23781,0.00038,0.00076,0.62722,0.00083,0.00000\n406,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n407,0.00217,0.33628,0.00003,0.00255,0.00044,0.00542,0.08651,0.05473,0.51187\n408,0.00002,0.32325,0.00073,0.00013,0.00012,0.00014,0.67515,0.00046,0.00000\n409,0.00120,0.66028,0.00005,0.00092,0.16775,0.00133,0.15561,0.01281,0.00005\n410,0.00019,0.19557,0.00049,0.00051,0.00107,0.02469,0.77113,0.00631,0.00004\n411,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n412,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n413,0.29255,0.00225,0.07972,0.61200,0.00777,0.00178,0.00390,0.00002,0.00001\n414,0.55665,0.00363,0.00006,0.00903,0.07021,0.35911,0.00120,0.00011,0.00000\n415,0.01384,0.83701,0.00699,0.01170,0.01296,0.01016,0.09868,0.00417,0.00449\n416,0.02326,0.01161,0.01314,0.06047,0.83385,0.03335,0.01260,0.00600,0.00572\n417,0.65916,0.19186,0.00000,0.00038,0.00011,0.00006,0.14810,0.00032,0.00000\n418,0.89097,0.01652,0.00137,0.08131,0.00405,0.00042,0.00441,0.00088,0.00007\n419,0.00001,0.01646,0.00000,0.00000,0.00002,0.00002,0.98286,0.00062,0.00001\n420,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n421,0.81085,0.01920,0.00068,0.03692,0.09507,0.01930,0.01724,0.00059,0.00015\n422,0.54194,0.32643,0.00001,0.03512,0.00034,0.00021,0.03117,0.06421,0.00057\n423,0.22803,0.55131,0.00142,0.01568,0.00406,0.00292,0.18814,0.00775,0.00069\n424,0.00001,0.99801,0.00000,0.00003,0.00001,0.00003,0.00191,0.00000,0.00000\n425,0.02774,0.00969,0.32438,0.01498,0.03062,0.53945,0.05302,0.00012,0.00000\n426,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n427,0.62379,0.05683,0.01359,0.11070,0.01961,0.02625,0.11689,0.01558,0.01676\n428,0.69848,0.02329,0.01482,0.04483,0.03960,0.13605,0.02348,0.00938,0.01006\n429,0.02963,0.00003,0.01362,0.42969,0.50682,0.02009,0.00012,0.00000,0.00000\n430,0.03925,0.00381,0.05295,0.76587,0.08794,0.03100,0.01851,0.00044,0.00023\n431,0.06019,0.29608,0.00055,0.06391,0.01436,0.02980,0.53090,0.00419,0.00002\n432,0.01899,0.07154,0.00742,0.01806,0.02208,0.01477,0.83577,0.00593,0.00544\n433,0.00093,0.23680,0.00178,0.00242,0.00565,0.00134,0.74833,0.00259,0.00016\n434,0.58048,0.05383,0.00004,0.35644,0.00138,0.00065,0.00429,0.00266,0.00023\n435,0.04350,0.29638,0.01246,0.05322,0.03567,0.07728,0.47816,0.00323,0.00009\n436,0.00124,0.32143,0.00058,0.00225,0.00216,0.00083,0.66883,0.00254,0.00014\n437,0.00001,0.02158,0.00015,0.04920,0.00001,0.00362,0.92540,0.00003,0.00000\n438,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n439,0.00373,0.40854,0.00722,0.00394,0.02184,0.00501,0.54733,0.00218,0.00021\n440,0.00012,0.04114,0.00001,0.00004,0.00007,0.00000,0.43400,0.01448,0.51014\n441,0.31273,0.01539,0.00373,0.57422,0.06135,0.00385,0.02867,0.00004,0.00002\n442,0.00002,0.03161,0.00003,0.00004,0.00004,0.00001,0.64682,0.00635,0.31508\n443,0.00956,0.22606,0.00011,0.13714,0.00035,0.00064,0.62498,0.00115,0.00001\n444,0.72503,0.00083,0.09803,0.14206,0.00123,0.00099,0.03182,0.00000,0.00000\n445,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n446,0.57101,0.00035,0.00000,0.03667,0.00000,0.00000,0.39197,0.00000,0.00000\n447,0.01150,0.14109,0.00173,0.82462,0.00077,0.00051,0.01897,0.00068,0.00013\n448,0.21726,0.14221,0.00018,0.58512,0.00038,0.00116,0.03125,0.02214,0.00030\n449,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n450,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n451,0.00791,0.00045,0.00001,0.19064,0.00000,0.00000,0.80099,0.00000,0.00000\n452,0.04408,0.00545,0.00055,0.02446,0.00622,0.00023,0.91845,0.00045,0.00011\n453,0.04508,0.00427,0.00169,0.87968,0.00133,0.00090,0.06682,0.00017,0.00006\n454,0.03988,0.21260,0.01088,0.05062,0.02528,0.02492,0.61549,0.01086,0.00947\n455,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n456,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n457,0.19725,0.04018,0.00006,0.01609,0.00047,0.00013,0.74416,0.00159,0.00007\n458,0.00907,0.22975,0.00007,0.74246,0.00005,0.00001,0.01855,0.00004,0.00000\n459,0.01645,0.34737,0.00188,0.03491,0.04194,0.06394,0.48212,0.01127,0.00011\n460,0.77980,0.00286,0.05669,0.14517,0.00485,0.00067,0.00981,0.00008,0.00007\n461,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n462,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n463,0.01806,0.24057,0.00486,0.07488,0.02393,0.04103,0.59276,0.00389,0.00002\n464,0.00375,0.28780,0.00404,0.06290,0.02172,0.02212,0.59065,0.00676,0.00026\n465,0.46909,0.38566,0.00001,0.04168,0.00044,0.00031,0.02914,0.07328,0.00039\n466,0.58872,0.02306,0.00088,0.20657,0.01866,0.15843,0.00176,0.00180,0.00012\n467,0.66023,0.02068,0.00034,0.01627,0.15378,0.13597,0.01244,0.00018,0.00011\n468,0.23528,0.05798,0.00010,0.00674,0.00446,0.00024,0.69063,0.00432,0.00025\n469,0.38648,0.31154,0.00001,0.03140,0.00036,0.00019,0.21548,0.05390,0.00065\n470,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n471,0.73914,0.06407,0.01187,0.07416,0.02318,0.02373,0.04307,0.01144,0.00934\n472,0.00001,0.07000,0.00003,0.00002,0.00009,0.00002,0.92937,0.00046,0.00000\n473,0.00016,0.36653,0.00013,0.00032,0.00083,0.00013,0.63041,0.00149,0.00000\n474,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n475,0.15814,0.00335,0.03632,0.17632,0.14891,0.01639,0.46055,0.00001,0.00001\n476,0.03988,0.21260,0.01088,0.05062,0.02528,0.02492,0.61549,0.01086,0.00947\n477,0.03394,0.00032,0.00008,0.67573,0.00002,0.00003,0.28988,0.00000,0.00000\n478,0.00617,0.00038,0.53814,0.25494,0.07701,0.12249,0.00087,0.00000,0.00000\n479,0.05987,0.10612,0.00043,0.81999,0.00021,0.00136,0.01167,0.00035,0.00000\n480,0.12089,0.00036,0.00193,0.87472,0.00061,0.00100,0.00037,0.00005,0.00007\n481,0.01944,0.01967,0.00060,0.30049,0.00041,0.00223,0.65709,0.00006,0.00001\n482,0.00012,0.32477,0.00055,0.00051,0.00350,0.00012,0.66906,0.00137,0.00000\n483,0.29536,0.01038,0.00001,0.00012,0.00001,0.00039,0.69368,0.00005,0.00000\n484,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n485,0.00958,0.09055,0.01041,0.00819,0.17386,0.01031,0.69041,0.00653,0.00016\n486,0.22116,0.00086,0.00018,0.77679,0.00052,0.00008,0.00038,0.00001,0.00002\n487,0.00305,0.14218,0.00235,0.00330,0.01019,0.00378,0.83107,0.00387,0.00021\n488,0.26979,0.02102,0.00073,0.00424,0.00267,0.00313,0.67020,0.02518,0.00304\n489,0.07233,0.01945,0.06416,0.23551,0.08624,0.48097,0.01902,0.01145,0.01086\n490,0.04176,0.01603,0.03433,0.13815,0.71843,0.02045,0.01527,0.00825,0.00733\n491,0.00076,0.00084,0.00004,0.29857,0.00000,0.00001,0.69978,0.00000,0.00000\n492,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n493,0.03614,0.18418,0.00582,0.70225,0.00255,0.00160,0.06576,0.00154,0.00016\n494,0.52358,0.37105,0.00002,0.04319,0.00020,0.00046,0.06011,0.00127,0.00012\n495,0.00614,0.39468,0.00053,0.00423,0.00154,0.00160,0.57048,0.01880,0.00200\n496,0.33004,0.00007,0.00014,0.66936,0.00011,0.00015,0.00010,0.00000,0.00003\n497,0.13398,0.00146,0.00512,0.04925,0.19384,0.22368,0.39227,0.00026,0.00014\n498,0.00014,0.34764,0.00002,0.00046,0.00055,0.00041,0.64928,0.00149,0.00001\n499,0.00045,0.69089,0.00001,0.21345,0.00007,0.00002,0.09504,0.00007,0.00000\n500,0.01359,0.02795,0.00514,0.01017,0.01086,0.01114,0.91179,0.00486,0.00450\n501,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n502,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n503,0.00001,0.15366,0.00001,0.00004,0.00027,0.00049,0.84495,0.00057,0.00000\n504,0.14213,0.06689,0.00728,0.32599,0.02986,0.01677,0.40332,0.00726,0.00050\n505,0.00085,0.55406,0.00126,0.00362,0.00425,0.00408,0.42711,0.00453,0.00024\n506,0.64299,0.00211,0.00355,0.14394,0.15664,0.05037,0.00029,0.00009,0.00002\n507,0.01256,0.14984,0.01180,0.02321,0.07093,0.60763,0.10254,0.02026,0.00123\n508,0.14446,0.01069,0.00004,0.83231,0.00002,0.00020,0.01177,0.00046,0.00005\n509,0.65783,0.00021,0.00000,0.02400,0.00000,0.00000,0.31796,0.00000,0.00000\n510,0.46803,0.15920,0.01403,0.07751,0.02564,0.02160,0.17347,0.02681,0.03372\n511,0.54247,0.25837,0.00002,0.00617,0.00100,0.00431,0.17603,0.01055,0.00108\n512,0.00003,0.80423,0.00000,0.00003,0.00014,0.00008,0.19526,0.00023,0.00000\n513,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n514,0.54474,0.00075,0.00210,0.44114,0.00720,0.00215,0.00160,0.00017,0.00015\n515,0.05831,0.01502,0.00010,0.91889,0.00004,0.00002,0.00759,0.00003,0.00000\n516,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n517,0.82217,0.00022,0.00007,0.00229,0.17457,0.00066,0.00002,0.00000,0.00000\n518,0.52351,0.33114,0.00002,0.06900,0.00037,0.00013,0.02699,0.04830,0.00054\n519,0.15011,0.45075,0.00019,0.02736,0.00563,0.01212,0.32911,0.02456,0.00018\n520,0.50193,0.00002,0.00002,0.47300,0.00000,0.00001,0.02502,0.00000,0.00000\n521,0.08442,0.49465,0.01806,0.06793,0.07278,0.02808,0.21293,0.01131,0.00984\n522,0.25224,0.17488,0.01696,0.06108,0.02083,0.03022,0.41694,0.01448,0.01238\n523,0.00137,0.35975,0.00196,0.00384,0.00775,0.00172,0.61901,0.00436,0.00024\n524,0.00288,0.04859,0.00009,0.55081,0.00002,0.00002,0.39751,0.00008,0.00000\n525,0.00464,0.01431,0.00010,0.60108,0.00001,0.00007,0.37979,0.00000,0.00000\n526,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000\n527,0.89354,0.00006,0.00004,0.10609,0.00016,0.00006,0.00004,0.00000,0.00001\n528,0.00003,0.12192,0.00009,0.00010,0.00159,0.00014,0.87502,0.00111,0.00000\n529,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n530,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n531,0.91081,0.00291,0.00014,0.07986,0.00014,0.00058,0.00081,0.00466,0.00009\n532,0.01301,0.54599,0.00290,0.14729,0.00352,0.01737,0.26486,0.00445,0.00061\n533,0.00099,0.15956,0.00010,0.00113,0.00028,0.00025,0.83468,0.00300,0.00001\n534,0.69570,0.03320,0.00001,0.00035,0.00001,0.00001,0.27072,0.00000,0.00000\n535,0.70903,0.00177,0.00029,0.00750,0.27754,0.00226,0.00145,0.00006,0.00010\n536,0.47869,0.38138,0.00001,0.04261,0.00046,0.00024,0.02797,0.06824,0.00041\n537,0.08507,0.04494,0.00059,0.00882,0.21956,0.00192,0.06097,0.03335,0.54478\n538,0.03155,0.23096,0.00160,0.10773,0.02122,0.01829,0.58814,0.00050,0.00001\n539,0.00195,0.20229,0.00452,0.07352,0.01491,0.11048,0.59176,0.00057,0.00000\n540,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n541,0.33049,0.59055,0.00000,0.01805,0.00018,0.00016,0.01591,0.04431,0.00035\n542,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n543,0.08042,0.07460,0.01397,0.08717,0.04613,0.02981,0.64021,0.01351,0.01418\n544,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n545,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n546,0.07626,0.51157,0.00456,0.06258,0.00465,0.01625,0.29953,0.02379,0.00081\n547,0.00806,0.39476,0.01478,0.01160,0.08228,0.04503,0.43436,0.00858,0.00056\n548,0.44289,0.05265,0.00000,0.00352,0.00182,0.00265,0.01799,0.07380,0.40467\n549,0.20338,0.00414,0.10463,0.67307,0.00690,0.00239,0.00548,0.00001,0.00000\n550,0.19872,0.22338,0.01301,0.12780,0.04054,0.01939,0.37112,0.00559,0.00045\n551,0.02793,0.00337,0.00000,0.00248,0.00066,0.01314,0.16155,0.26161,0.52926\n552,0.20013,0.77413,0.00000,0.00003,0.00000,0.00000,0.02570,0.00001,0.00000\n553,0.00085,0.39511,0.00155,0.00666,0.02978,0.10580,0.45448,0.00577,0.00000\n554,0.42385,0.31607,0.00038,0.01068,0.00162,0.00188,0.24052,0.00449,0.00051\n555,0.02198,0.02868,0.16502,0.03057,0.02620,0.01341,0.70023,0.00696,0.00695\n556,0.00006,0.75246,0.00001,0.00046,0.00040,0.00021,0.24594,0.00046,0.00000\n557,0.00017,0.02793,0.45105,0.00090,0.00772,0.00050,0.51173,0.00000,0.00000\n558,0.00120,0.74900,0.00015,0.18621,0.00406,0.00531,0.05376,0.00031,0.00000\n559,0.00077,0.01559,0.00000,0.00043,0.00060,0.94832,0.00238,0.03191,0.00000\n560,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n561,0.51114,0.01581,0.00939,0.05396,0.35232,0.01514,0.02693,0.00806,0.00725\n562,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n563,0.06387,0.02847,0.00064,0.23017,0.00807,0.00026,0.66830,0.00021,0.00000\n564,0.37500,0.00387,0.00038,0.61661,0.00138,0.00205,0.00068,0.00002,0.00001\n565,0.00002,0.05117,0.00009,0.00045,0.00005,0.00010,0.94805,0.00007,0.00000\n566,0.35014,0.06284,0.00010,0.04410,0.00180,0.02512,0.51568,0.00019,0.00003\n567,0.00699,0.87040,0.00000,0.00406,0.00055,0.00018,0.02029,0.01551,0.08202\n568,0.00072,0.42533,0.00001,0.02705,0.00004,0.00015,0.02441,0.52216,0.00013\n569,0.08622,0.08253,0.00730,0.00534,0.02011,0.00455,0.77110,0.02112,0.00173\n570,0.68190,0.27608,0.00002,0.02603,0.00003,0.00016,0.01560,0.00011,0.00007\n571,0.00015,0.01166,0.00000,0.00021,0.00003,0.00000,0.00176,0.05361,0.93258\n572,0.00001,0.07179,0.00005,0.00003,0.00012,0.00022,0.92770,0.00008,0.00000\n573,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n574,0.02090,0.01281,0.00005,0.18391,0.00002,0.00005,0.78218,0.00008,0.00000\n575,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n576,0.97512,0.00175,0.00004,0.00480,0.00005,0.00100,0.01715,0.00007,0.00002\n577,0.03569,0.04560,0.00925,0.00524,0.62191,0.00028,0.28026,0.00150,0.00027\n578,0.06010,0.05036,0.00436,0.63539,0.01965,0.18154,0.03942,0.00837,0.00081\n579,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n580,0.32276,0.00148,0.00018,0.67437,0.00031,0.00022,0.00066,0.00001,0.00001\n581,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n582,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n583,0.79760,0.05376,0.00671,0.02318,0.01130,0.01462,0.05537,0.00907,0.02839\n584,0.12050,0.00725,0.00012,0.51026,0.00009,0.00005,0.36171,0.00002,0.00000\n585,0.00679,0.00239,0.00061,0.35359,0.32979,0.00031,0.11947,0.01753,0.16953\n586,0.62855,0.02308,0.01465,0.04615,0.04337,0.20125,0.02353,0.00937,0.01005\n587,0.04325,0.01926,0.00576,0.23911,0.00383,0.00141,0.68259,0.00403,0.00076\n588,0.00117,0.00832,0.00005,0.63538,0.00001,0.00001,0.35505,0.00001,0.00000\n589,0.00017,0.01482,0.00011,0.00022,0.00033,0.00007,0.98391,0.00034,0.00003\n590,0.00026,0.36061,0.03691,0.00059,0.02616,0.00230,0.56774,0.00541,0.00002\n591,0.61984,0.12841,0.01306,0.09516,0.01809,0.02305,0.06739,0.02115,0.01384\n592,0.18178,0.00940,0.00007,0.42732,0.00003,0.00004,0.38107,0.00018,0.00010\n593,0.34978,0.01806,0.00140,0.61416,0.00154,0.00037,0.01416,0.00049,0.00004\n594,0.00001,0.07328,0.00001,0.00002,0.00004,0.00001,0.92633,0.00030,0.00000\n595,0.00034,0.30854,0.00041,0.00696,0.00045,0.00025,0.68197,0.00108,0.00000\n596,0.68455,0.29928,0.00000,0.00064,0.00005,0.00002,0.01542,0.00003,0.00001\n597,0.13886,0.02190,0.00827,0.76655,0.01575,0.01368,0.02218,0.00622,0.00659\n598,0.01134,0.23148,0.00001,0.29125,0.00008,0.00002,0.46526,0.00056,0.00000\n599,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n600,0.00268,0.31239,0.00224,0.00341,0.01166,0.00269,0.66033,0.00436,0.00024\n601,0.17180,0.44314,0.00022,0.02768,0.00312,0.00502,0.32139,0.02742,0.00021\n602,0.47582,0.36740,0.00002,0.05066,0.00040,0.00016,0.04618,0.05889,0.00048\n603,0.64360,0.00137,0.00125,0.07211,0.14288,0.13871,0.00007,0.00000,0.00001\n604,0.00098,0.54581,0.00001,0.00222,0.00045,0.00020,0.44881,0.00152,0.00000\n605,0.00004,0.96367,0.00001,0.00015,0.00007,0.01829,0.01762,0.00015,0.00000\n606,0.00883,0.00186,0.00270,0.94747,0.00052,0.00134,0.03643,0.00057,0.00028\n607,0.26506,0.00044,0.00008,0.67795,0.00000,0.00010,0.05634,0.00003,0.00000\n608,0.00002,0.59862,0.00000,0.00003,0.00003,0.00002,0.40120,0.00008,0.00000\n609,0.01223,0.00726,0.00001,0.39269,0.00003,0.00000,0.00064,0.03977,0.54737\n610,0.21865,0.00923,0.00022,0.00019,0.00002,0.00069,0.77098,0.00001,0.00001\n611,0.08931,0.16009,0.00201,0.08016,0.17710,0.00231,0.44800,0.03244,0.00859\n612,0.00002,0.07372,0.00001,0.00002,0.00009,0.00003,0.92458,0.00149,0.00004\n613,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n614,0.03624,0.56216,0.01534,0.03109,0.03517,0.02534,0.27577,0.00954,0.00935\n615,0.05272,0.13198,0.01471,0.08259,0.03053,0.02071,0.64610,0.01124,0.00942\n616,0.00001,0.05459,0.36840,0.00016,0.00958,0.00097,0.56628,0.00001,0.00000\n617,0.00451,0.05267,0.15143,0.01547,0.07427,0.00580,0.69335,0.00229,0.00021\n618,0.39217,0.04925,0.01590,0.32732,0.08009,0.02365,0.10484,0.00629,0.00049\n619,0.00231,0.00364,0.00051,0.57840,0.00034,0.40531,0.00768,0.00181,0.00000\n620,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n621,0.21267,0.69696,0.00033,0.00340,0.00133,0.00135,0.08155,0.00222,0.00019\n622,0.00122,0.02600,0.28960,0.00143,0.11501,0.00066,0.56600,0.00008,0.00000\n623,0.01384,0.83701,0.00699,0.01170,0.01296,0.01016,0.09868,0.00417,0.00449\n624,0.03316,0.00407,0.01072,0.13856,0.05420,0.01577,0.74318,0.00024,0.00010\n625,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n626,0.47308,0.22013,0.01606,0.12113,0.10861,0.00390,0.02119,0.03550,0.00039\n627,0.59988,0.01426,0.00000,0.00020,0.00002,0.00005,0.38555,0.00002,0.00002\n628,0.48561,0.36911,0.00001,0.03534,0.00036,0.00021,0.02900,0.07966,0.00071\n629,0.04883,0.11260,0.01387,0.05522,0.02253,0.60147,0.12574,0.01048,0.00926\n630,0.00017,0.20323,0.00001,0.00006,0.00044,0.00007,0.78693,0.00906,0.00003\n631,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n632,0.28599,0.41297,0.00198,0.02468,0.01672,0.01248,0.19294,0.04850,0.00374\n633,0.70609,0.00863,0.00197,0.02667,0.02023,0.02354,0.21111,0.00145,0.00031\n634,0.11788,0.02107,0.01487,0.43139,0.30199,0.05983,0.03580,0.00837,0.00880\n635,0.00050,0.13203,0.00059,0.00097,0.00241,0.00045,0.86107,0.00185,0.00013\n636,0.50868,0.01469,0.01106,0.03641,0.37903,0.01794,0.01733,0.00741,0.00745\n637,0.56309,0.19891,0.00008,0.07163,0.00256,0.15810,0.00359,0.00202,0.00002\n638,0.00427,0.04706,0.00518,0.83731,0.00062,0.02151,0.08404,0.00001,0.00000\n639,0.01814,0.00460,0.00029,0.87039,0.00195,0.00040,0.10423,0.00000,0.00000\n640,0.38361,0.42368,0.00191,0.06187,0.00505,0.00669,0.09729,0.01732,0.00259\n641,0.04591,0.01163,0.00003,0.94096,0.00001,0.00004,0.00124,0.00018,0.00000\n642,0.28717,0.00010,0.00007,0.71230,0.00020,0.00005,0.00011,0.00000,0.00000\n643,0.54405,0.30791,0.00004,0.07389,0.00043,0.00014,0.02640,0.04651,0.00063\n644,0.02953,0.10894,0.00909,0.02957,0.01784,0.01963,0.75959,0.01265,0.01316\n645,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n646,0.18894,0.78529,0.00001,0.00012,0.00000,0.00002,0.00599,0.00311,0.01652\n647,0.66563,0.01901,0.00000,0.00020,0.00001,0.00001,0.31386,0.00052,0.00076\n648,0.73260,0.02359,0.00884,0.14899,0.02866,0.01497,0.02439,0.00837,0.00959\n649,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n650,0.03103,0.01389,0.00177,0.52421,0.00147,0.00121,0.42587,0.00042,0.00013\n651,0.06010,0.06418,0.00127,0.00847,0.62190,0.00038,0.24135,0.00207,0.00028\n652,0.09135,0.00275,0.00011,0.90533,0.00005,0.00002,0.00039,0.00000,0.00000\n653,0.65649,0.00173,0.00237,0.22600,0.00152,0.11169,0.00019,0.00001,0.00000\n654,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n655,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n656,0.26454,0.00413,0.00008,0.73094,0.00000,0.00005,0.00026,0.00000,0.00000\n657,0.02039,0.08891,0.01269,0.03563,0.01865,0.03130,0.77328,0.01036,0.00879\n658,0.36567,0.00407,0.00071,0.58563,0.00601,0.02806,0.00058,0.00571,0.00356\n659,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n660,0.51878,0.33871,0.00001,0.05544,0.00040,0.00018,0.02715,0.05868,0.00065\n661,0.00034,0.00666,0.00008,0.43784,0.00001,0.00001,0.55506,0.00000,0.00000\n662,0.00304,0.77833,0.00064,0.00570,0.00342,0.00247,0.17203,0.02929,0.00508\n663,0.80465,0.01788,0.00837,0.10613,0.01344,0.01306,0.02328,0.00578,0.00741\n664,0.00001,0.37255,0.00000,0.00001,0.00005,0.00003,0.62722,0.00013,0.00000\n665,0.99992,0.00000,0.00000,0.00008,0.00000,0.00000,0.00000,0.00000,0.00000\n666,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n667,0.00007,0.86376,0.00000,0.00010,0.00020,0.00007,0.13523,0.00057,0.00000\n668,0.50093,0.35293,0.00005,0.04820,0.00051,0.00051,0.02798,0.06821,0.00068\n669,0.00965,0.04070,0.01098,0.03863,0.08543,0.00013,0.81380,0.00046,0.00022\n670,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n671,0.46755,0.00112,0.00000,0.00333,0.00001,0.00002,0.52793,0.00002,0.00002\n672,0.83834,0.00138,0.00005,0.14041,0.00030,0.00013,0.00060,0.00042,0.01837\n673,0.03205,0.01741,0.00260,0.34803,0.00287,0.01745,0.57561,0.00362,0.00036\n674,0.01128,0.91702,0.00450,0.01514,0.01304,0.00586,0.02643,0.00337,0.00336\n675,0.00131,0.38260,0.00007,0.53916,0.00174,0.03018,0.04406,0.00088,0.00000\n676,0.01912,0.06997,0.00307,0.08453,0.02309,0.04779,0.75201,0.00042,0.00000\n677,0.17645,0.00007,0.00910,0.38931,0.41278,0.01218,0.00009,0.00000,0.00001\n678,0.03773,0.00047,0.00002,0.55106,0.00001,0.00001,0.41070,0.00000,0.00000\n679,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n680,0.58714,0.00106,0.00021,0.40353,0.00428,0.00304,0.00069,0.00001,0.00004\n681,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n682,0.30246,0.01099,0.00000,0.00029,0.00001,0.00036,0.68589,0.00000,0.00000\n683,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n684,0.06836,0.00016,0.00003,0.68213,0.00002,0.00001,0.24929,0.00000,0.00000\n685,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n686,0.00862,0.88488,0.00006,0.01496,0.00010,0.01935,0.06985,0.00216,0.00002\n687,0.69323,0.26623,0.00000,0.00195,0.00021,0.00019,0.03542,0.00270,0.00007\n688,0.54285,0.01121,0.00013,0.04010,0.00290,0.00161,0.40066,0.00051,0.00003\n689,0.67171,0.00154,0.00000,0.00009,0.00000,0.00001,0.32665,0.00000,0.00000\n690,0.25019,0.00634,0.00082,0.70824,0.00113,0.01984,0.01338,0.00004,0.00002\n691,0.03806,0.36449,0.00648,0.20609,0.00589,0.03755,0.24359,0.08121,0.01663\n692,0.17331,0.17822,0.01957,0.39106,0.04114,0.06888,0.10366,0.01204,0.01211\n693,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n694,0.56249,0.00039,0.00008,0.43652,0.00005,0.00002,0.00045,0.00000,0.00000\n695,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n696,0.51224,0.00125,0.00001,0.04892,0.00018,0.00448,0.43286,0.00003,0.00003\n697,0.25506,0.03476,0.00448,0.02670,0.01291,0.00658,0.64449,0.01343,0.00159\n698,0.00003,0.93220,0.00000,0.00006,0.00016,0.00009,0.06647,0.00099,0.00000\n699,0.08466,0.20463,0.01261,0.04530,0.02358,0.03591,0.57444,0.00925,0.00962\n700,0.00001,0.01424,0.00001,0.00001,0.00001,0.00001,0.98570,0.00001,0.00000\n701,0.01760,0.01486,0.00004,0.58571,0.00001,0.00002,0.38175,0.00001,0.00000\n702,0.05308,0.02346,0.00001,0.00172,0.00002,0.00000,0.92158,0.00006,0.00007\n703,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n704,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n705,0.36862,0.00057,0.00010,0.63023,0.00015,0.00005,0.00028,0.00000,0.00000\n706,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n707,0.05553,0.01425,0.03817,0.74152,0.07654,0.03603,0.03497,0.00248,0.00051\n708,0.34179,0.04610,0.01056,0.17584,0.08524,0.02450,0.31499,0.00074,0.00024\n709,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n710,0.20395,0.40620,0.00031,0.03555,0.00391,0.00546,0.31696,0.02740,0.00026\n711,0.57668,0.02375,0.00358,0.10270,0.02582,0.26232,0.00510,0.00004,0.00001\n712,0.00596,0.00990,0.00000,0.00161,0.00142,0.00003,0.01175,0.14032,0.82901\n713,0.04353,0.17698,0.01386,0.04958,0.02205,0.03740,0.63650,0.00970,0.01040\n714,0.02355,0.06832,0.00192,0.00919,0.05495,0.00017,0.83976,0.00201,0.00013\n715,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n716,0.04495,0.00907,0.05315,0.22818,0.11865,0.05874,0.48645,0.00056,0.00024\n717,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n718,0.05142,0.08993,0.01488,0.02798,0.03357,0.02090,0.74689,0.00713,0.00730\n719,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n720,0.01690,0.05185,0.00008,0.04013,0.00007,0.00134,0.88862,0.00080,0.00021\n721,0.08812,0.11669,0.00143,0.74027,0.00059,0.00045,0.05187,0.00045,0.00013\n722,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n723,0.02198,0.02868,0.16502,0.03057,0.02620,0.01341,0.70023,0.00696,0.00695\n724,0.31210,0.04534,0.00005,0.63336,0.00002,0.00012,0.00868,0.00032,0.00001\n725,0.71754,0.02893,0.00000,0.00027,0.00000,0.00006,0.22475,0.00519,0.02326\n726,0.03141,0.02626,0.00015,0.77966,0.00035,0.00004,0.15847,0.00156,0.00210\n727,0.08277,0.13461,0.00044,0.03085,0.00048,0.00240,0.74786,0.00057,0.00002\n728,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n729,0.38669,0.00178,0.00601,0.57778,0.01882,0.00556,0.00221,0.00084,0.00031\n730,0.48638,0.19299,0.01653,0.15294,0.07033,0.00415,0.04160,0.03456,0.00053\n731,0.02015,0.02207,0.00013,0.45179,0.00003,0.00002,0.50580,0.00001,0.00000\n732,0.00019,0.02252,0.00005,0.00106,0.00004,0.26944,0.70661,0.00009,0.00000\n733,0.03532,0.01259,0.00065,0.64731,0.00004,0.00165,0.30234,0.00007,0.00003\n734,0.00046,0.01404,0.02712,0.00028,0.03994,0.00005,0.91777,0.00032,0.00002\n735,0.00976,0.19030,0.00643,0.01453,0.04382,0.55495,0.15605,0.02298,0.00118\n736,0.00004,0.35087,0.00001,0.00027,0.00034,0.00040,0.64278,0.00511,0.00018\n737,0.16816,0.40560,0.01701,0.04126,0.04258,0.04039,0.24107,0.03011,0.01382\n738,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n739,0.02598,0.18347,0.00027,0.28981,0.00028,0.00490,0.49507,0.00022,0.00000\n740,0.00002,0.22208,0.00006,0.00005,0.00043,0.00012,0.77655,0.00069,0.00000\n741,0.00120,0.25698,0.00195,0.00251,0.00716,0.00126,0.72610,0.00268,0.00016\n742,0.00094,0.83837,0.00001,0.00399,0.00029,0.00007,0.15574,0.00059,0.00000\n743,0.05297,0.00033,0.00003,0.94660,0.00001,0.00000,0.00006,0.00000,0.00000\n744,0.00002,0.23601,0.00005,0.00033,0.00025,0.00009,0.76282,0.00043,0.00000\n745,0.01039,0.00700,0.53048,0.02270,0.04173,0.00122,0.00789,0.02449,0.35410\n746,0.18163,0.05820,0.00014,0.14396,0.00038,0.00058,0.61439,0.00071,0.00001\n747,0.46872,0.00665,0.00009,0.49994,0.00025,0.01087,0.01326,0.00021,0.00002\n748,0.28651,0.01007,0.00000,0.00878,0.00001,0.00002,0.68773,0.00019,0.00669\n749,0.80413,0.00397,0.00074,0.15867,0.00358,0.00456,0.00468,0.01696,0.00271\n750,0.00318,0.03929,0.00002,0.71400,0.00001,0.00000,0.24347,0.00003,0.00000\n751,0.00495,0.00470,0.00307,0.39475,0.00584,0.00128,0.58514,0.00022,0.00005\n752,0.43872,0.01245,0.00882,0.03596,0.44518,0.03135,0.01405,0.00687,0.00659\n753,0.00043,0.78354,0.00001,0.01653,0.00008,0.00002,0.19889,0.00050,0.00000\n754,0.24410,0.02020,0.04475,0.67815,0.00463,0.00660,0.00131,0.00025,0.00001\n755,0.91557,0.01653,0.00098,0.02549,0.00770,0.01002,0.00987,0.01224,0.00160\n756,0.00062,0.20747,0.00084,0.00146,0.00374,0.00144,0.78150,0.00277,0.00016\n757,0.08189,0.00157,0.88324,0.02406,0.00377,0.00027,0.00520,0.00000,0.00000\n758,0.18375,0.03730,0.02215,0.65804,0.02109,0.02588,0.03203,0.00971,0.01005\n759,0.01809,0.00755,0.00421,0.01322,0.01487,0.93100,0.00582,0.00265,0.00259\n760,0.00046,0.93350,0.00020,0.00142,0.00113,0.00085,0.06023,0.00202,0.00019\n761,0.02627,0.02410,0.00062,0.00150,0.00132,0.00007,0.94603,0.00008,0.00001\n762,0.07582,0.02777,0.02078,0.27962,0.18151,0.02732,0.38710,0.00008,0.00001\n763,0.55618,0.00081,0.00202,0.43007,0.00667,0.00233,0.00159,0.00018,0.00015\n764,0.00601,0.65556,0.00007,0.00294,0.00056,0.00006,0.33236,0.00235,0.00009\n765,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n766,0.00582,0.48590,0.00124,0.00468,0.33504,0.01088,0.14943,0.00699,0.00002\n767,0.07680,0.22456,0.01156,0.08720,0.02472,0.02854,0.52533,0.01025,0.01104\n768,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n769,0.00055,0.38871,0.00062,0.00183,0.00826,0.00875,0.58802,0.00325,0.00001\n770,0.33855,0.01450,0.00021,0.61671,0.00253,0.00444,0.02269,0.00027,0.00010\n771,0.02232,0.61869,0.00999,0.01821,0.01498,0.02791,0.27573,0.00670,0.00547\n772,0.00141,0.39602,0.00052,0.00214,0.00200,0.00214,0.58402,0.01106,0.00069\n773,0.78610,0.01631,0.00750,0.13443,0.01623,0.01203,0.01660,0.00538,0.00542\n774,0.43820,0.51547,0.00001,0.02565,0.00001,0.00929,0.01133,0.00004,0.00000\n775,0.43872,0.01245,0.00882,0.03596,0.44518,0.03135,0.01405,0.00687,0.00659\n776,0.47661,0.34130,0.00193,0.02882,0.01885,0.01185,0.04830,0.06685,0.00549\n777,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n778,0.25783,0.20584,0.00005,0.03351,0.00055,0.00018,0.47173,0.03013,0.00018\n779,0.74252,0.01858,0.00023,0.03256,0.00208,0.00204,0.20134,0.00052,0.00013\n780,0.01713,0.25048,0.00019,0.00086,0.00301,0.00376,0.72137,0.00317,0.00003\n781,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n782,0.95196,0.00209,0.00000,0.04564,0.00003,0.00005,0.00016,0.00003,0.00004\n783,0.00362,0.04933,0.01000,0.00898,0.00323,0.00063,0.92414,0.00007,0.00000\n784,0.50868,0.01469,0.01106,0.03641,0.37903,0.01794,0.01733,0.00741,0.00745\n785,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n786,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n787,0.00234,0.02492,0.00007,0.59425,0.00004,0.00004,0.37827,0.00007,0.00000\n788,0.02996,0.27247,0.01200,0.03340,0.02613,0.04766,0.55777,0.01080,0.00981\n789,0.61748,0.05272,0.01837,0.13241,0.04674,0.04668,0.05754,0.01657,0.01149\n790,0.04907,0.24355,0.00037,0.43517,0.00020,0.01089,0.26009,0.00066,0.00001\n791,0.44560,0.01212,0.00000,0.01099,0.00001,0.00054,0.53037,0.00006,0.00031\n792,0.04472,0.17794,0.01446,0.04967,0.02545,0.03075,0.63727,0.01079,0.00895\n793,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n794,0.00838,0.03333,0.00883,0.02325,0.24313,0.01475,0.66812,0.00018,0.00003\n795,0.05483,0.57673,0.01578,0.05141,0.02913,0.03235,0.21625,0.01170,0.01182\n796,0.06871,0.00005,0.00005,0.93112,0.00003,0.00002,0.00002,0.00000,0.00000\n797,0.52140,0.30313,0.00202,0.02947,0.02068,0.01018,0.04890,0.05903,0.00519\n798,0.50165,0.35649,0.00001,0.03689,0.00042,0.00022,0.02865,0.07489,0.00078\n799,0.80465,0.01788,0.00837,0.10613,0.01344,0.01306,0.02328,0.00578,0.00741\n800,0.00374,0.32622,0.00583,0.00705,0.02601,0.02908,0.59340,0.00825,0.00042\n801,0.45806,0.39494,0.00001,0.03976,0.00052,0.00037,0.02686,0.07902,0.00046\n802,0.15858,0.07868,0.00410,0.04075,0.03431,0.64199,0.03203,0.00896,0.00060\n803,0.00158,0.69556,0.00362,0.00596,0.01147,0.00449,0.27190,0.00510,0.00032\n804,0.03023,0.00025,0.17199,0.78673,0.00649,0.00273,0.00158,0.00000,0.00000\n805,0.00021,0.40981,0.00002,0.00007,0.00012,0.00119,0.58791,0.00067,0.00000\n806,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n807,0.94309,0.00390,0.00030,0.04028,0.00035,0.00722,0.00479,0.00002,0.00005\n808,0.92457,0.03145,0.00018,0.00215,0.00005,0.00003,0.04153,0.00001,0.00003\n809,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n810,0.48953,0.00045,0.00010,0.50895,0.00058,0.00016,0.00022,0.00000,0.00001\n811,0.53059,0.00053,0.00055,0.00694,0.45946,0.00118,0.00070,0.00001,0.00004\n812,0.00198,0.63468,0.00117,0.01705,0.00316,0.00212,0.33534,0.00421,0.00029\n813,0.00017,0.86421,0.00000,0.00014,0.00025,0.00007,0.13327,0.00188,0.00001\n814,0.14518,0.00872,0.00286,0.63783,0.00883,0.00220,0.19399,0.00028,0.00011\n815,0.24593,0.73590,0.00000,0.00402,0.00007,0.00005,0.01384,0.00019,0.00000\n816,0.32268,0.29812,0.01270,0.05900,0.02828,0.03575,0.21962,0.01319,0.01066\n817,0.13026,0.02808,0.01387,0.72872,0.02611,0.01856,0.03794,0.00841,0.00805\n818,0.73996,0.01684,0.00177,0.21137,0.01279,0.00798,0.00442,0.00419,0.00068\n819,0.00448,0.39667,0.00024,0.00503,0.00230,0.01248,0.57324,0.00526,0.00030\n820,0.60956,0.00043,0.00010,0.00798,0.11508,0.26670,0.00014,0.00001,0.00000\n821,0.01900,0.03243,0.63036,0.02269,0.06675,0.00993,0.20605,0.00622,0.00657\n822,0.00097,0.05409,0.00005,0.00096,0.00096,0.00006,0.94244,0.00046,0.00001\n823,0.01128,0.91702,0.00450,0.01514,0.01304,0.00586,0.02643,0.00337,0.00336\n824,0.49872,0.01053,0.00005,0.46905,0.00011,0.00040,0.02015,0.00097,0.00002\n825,0.65999,0.02201,0.00033,0.01835,0.00142,0.00007,0.29650,0.00124,0.00009\n826,0.00088,0.14897,0.07937,0.00223,0.14411,0.00230,0.62115,0.00099,0.00000\n827,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n828,0.00047,0.03718,0.00007,0.00140,0.00037,0.92129,0.03882,0.00040,0.00000\n829,0.52272,0.00927,0.00003,0.00020,0.00006,0.00002,0.46770,0.00000,0.00000\n830,0.63488,0.00235,0.00001,0.09001,0.00000,0.00001,0.27272,0.00000,0.00002\n831,0.04006,0.00024,0.03157,0.06386,0.82058,0.04338,0.00026,0.00000,0.00005\n832,0.06828,0.25526,0.01635,0.07233,0.05329,0.02822,0.48605,0.01131,0.00890\n833,0.73916,0.00055,0.00039,0.25654,0.00282,0.00021,0.00032,0.00000,0.00001\n834,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n835,0.00012,0.00446,0.00000,0.00008,0.00009,0.00001,0.50913,0.03274,0.45337\n836,0.74351,0.00696,0.00137,0.23166,0.00595,0.00524,0.00318,0.00189,0.00024\n837,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n838,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n839,0.00750,0.03529,0.00184,0.93769,0.00436,0.00454,0.00872,0.00006,0.00000\n840,0.01177,0.03256,0.00050,0.00464,0.00109,0.00090,0.94721,0.00121,0.00012\n841,0.00464,0.77677,0.00001,0.00673,0.00029,0.00006,0.21111,0.00039,0.00000\n842,0.37433,0.00855,0.00001,0.01256,0.00001,0.56593,0.03853,0.00006,0.00002\n843,0.76354,0.04835,0.00000,0.02448,0.00003,0.00003,0.16346,0.00010,0.00001\n844,0.15065,0.22042,0.01143,0.03385,0.05833,0.08309,0.41126,0.02923,0.00174\n845,0.05084,0.17625,0.01608,0.05192,0.02369,0.02413,0.63649,0.01185,0.00875\n846,0.01376,0.24083,0.00312,0.02045,0.01522,0.00525,0.69715,0.00396,0.00026\n847,0.00003,0.43298,0.00003,0.00049,0.00035,0.00008,0.56577,0.00027,0.00000\n848,0.32680,0.19097,0.00012,0.04464,0.00165,0.06027,0.37337,0.00218,0.00001\n849,0.00051,0.13843,0.00061,0.00106,0.00245,0.00048,0.85448,0.00185,0.00013\n850,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n851,0.22643,0.02052,0.01101,0.12731,0.01887,0.55467,0.02650,0.00740,0.00729\n852,0.01625,0.21690,0.00246,0.09054,0.03759,0.12159,0.51456,0.00011,0.00000\n853,0.00403,0.10999,0.02064,0.00532,0.04721,0.00473,0.80427,0.00353,0.00028\n854,0.03291,0.00997,0.00013,0.61600,0.00014,0.00003,0.34078,0.00004,0.00000\n855,0.25314,0.03492,0.01012,0.67070,0.00507,0.00235,0.01032,0.00588,0.00750\n856,0.60437,0.34656,0.00787,0.00700,0.00253,0.01932,0.01192,0.00030,0.00013\n857,0.00602,0.13635,0.00066,0.70762,0.00089,0.00086,0.14757,0.00003,0.00000\n858,0.04689,0.04149,0.01230,0.59126,0.22266,0.02354,0.04655,0.00745,0.00786\n859,0.00002,0.06225,0.36468,0.00047,0.00722,0.00483,0.56052,0.00001,0.00000\n860,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n861,0.29410,0.31834,0.00046,0.05519,0.00223,0.00136,0.27929,0.04842,0.00060\n862,0.51273,0.34196,0.00001,0.04835,0.00053,0.00035,0.02741,0.06810,0.00056\n863,0.30912,0.01133,0.00145,0.02224,0.00023,0.00035,0.65486,0.00019,0.00023\n864,0.00227,0.02107,0.03001,0.00367,0.02955,0.00251,0.90974,0.00107,0.00011\n865,0.82867,0.00066,0.00062,0.16491,0.00259,0.00148,0.00081,0.00015,0.00011\n866,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n867,0.01216,0.03061,0.00738,0.06299,0.04780,0.00026,0.83845,0.00029,0.00006\n868,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n869,0.00669,0.05507,0.00002,0.00460,0.00008,0.00003,0.93320,0.00031,0.00000\n870,0.00016,0.53789,0.00004,0.00483,0.00061,0.00021,0.45533,0.00093,0.00000\n871,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n872,0.06629,0.65826,0.00093,0.01622,0.02739,0.03365,0.18258,0.01464,0.00004\n873,0.00006,0.61452,0.00000,0.00015,0.00027,0.00005,0.38399,0.00096,0.00000\n874,0.33135,0.45319,0.00008,0.17233,0.00095,0.00291,0.03836,0.00077,0.00007\n875,0.02725,0.06560,0.01245,0.02144,0.02250,0.01390,0.82112,0.00732,0.00842\n876,0.01102,0.41286,0.00014,0.54675,0.00002,0.00025,0.02895,0.00001,0.00000\n877,0.60682,0.06054,0.03074,0.16244,0.02330,0.03913,0.05162,0.01322,0.01219\n878,0.00067,0.54361,0.00030,0.00147,0.00671,0.00858,0.43592,0.00273,0.00001\n879,0.12489,0.00051,0.16154,0.62033,0.05693,0.03399,0.00122,0.00021,0.00038\n880,0.73754,0.03518,0.00024,0.11436,0.00405,0.10377,0.00455,0.00029,0.00002\n881,0.00030,0.70842,0.00000,0.00064,0.00031,0.00005,0.28908,0.00119,0.00001\n882,0.07465,0.25462,0.01748,0.05885,0.04539,0.07303,0.44648,0.01612,0.01337\n883,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n884,0.02358,0.00261,0.00547,0.03305,0.57400,0.00233,0.35879,0.00016,0.00001\n885,0.04158,0.23722,0.01280,0.04548,0.02412,0.03184,0.56217,0.03495,0.00984\n886,0.81439,0.00151,0.00034,0.00481,0.17500,0.00091,0.00291,0.00003,0.00010\n887,0.50868,0.01469,0.01106,0.03641,0.37903,0.01794,0.01733,0.00741,0.00745\n888,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n889,0.48790,0.35896,0.00001,0.04168,0.00052,0.00029,0.03274,0.07738,0.00051\n890,0.00294,0.35851,0.00058,0.00208,0.11853,0.00125,0.50331,0.01274,0.00006\n891,0.00022,0.35697,0.00001,0.00018,0.00038,0.00031,0.64094,0.00098,0.00001\n892,0.00001,0.09342,0.00001,0.00002,0.00006,0.00413,0.90125,0.00109,0.00001\n893,0.00006,0.25950,0.00005,0.00025,0.00095,0.00138,0.73673,0.00107,0.00001\n894,0.00002,0.02916,0.34073,0.00019,0.00819,0.00401,0.61769,0.00001,0.00000\n895,0.71572,0.05480,0.00787,0.09684,0.01359,0.01447,0.08122,0.00646,0.00903\n896,0.00603,0.00617,0.00112,0.64772,0.00037,0.00022,0.33812,0.00017,0.00008\n897,0.88090,0.05916,0.00000,0.02242,0.00129,0.00011,0.03474,0.00127,0.00011\n898,0.03492,0.15451,0.01069,0.02875,0.01602,0.01533,0.72252,0.00918,0.00808\n899,0.00010,0.89414,0.00000,0.00002,0.00004,0.00002,0.10402,0.00165,0.00001\n900,0.99841,0.00018,0.00000,0.00012,0.00012,0.00099,0.00003,0.00014,0.00001\n901,0.04103,0.22826,0.03315,0.06540,0.08096,0.06204,0.46037,0.01586,0.01292\n902,0.48288,0.25674,0.00241,0.09973,0.06196,0.00757,0.03638,0.05178,0.00055\n903,0.43872,0.01245,0.00882,0.03596,0.44518,0.03135,0.01405,0.00687,0.00659\n904,0.02120,0.00054,0.00432,0.24565,0.07933,0.00458,0.64437,0.00000,0.00000\n905,0.03994,0.22096,0.03233,0.02066,0.49764,0.00598,0.17661,0.00584,0.00004\n906,0.00369,0.00610,0.00014,0.00510,0.00023,0.00045,0.00349,0.03733,0.94347\n907,0.07117,0.17740,0.02309,0.05115,0.04264,0.07227,0.54201,0.01011,0.01016\n908,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n909,0.00180,0.57782,0.00189,0.00504,0.00742,0.00426,0.38708,0.01361,0.00108\n910,0.01641,0.02865,0.00416,0.30141,0.00566,0.00709,0.63535,0.00116,0.00011\n911,0.04779,0.01466,0.00022,0.36248,0.02585,0.51328,0.03415,0.00157,0.00000\n912,0.76160,0.01814,0.01017,0.08778,0.03875,0.01806,0.05011,0.00788,0.00751\n913,0.01875,0.18505,0.00053,0.59264,0.00064,0.00067,0.20163,0.00009,0.00000\n914,0.00040,0.15170,0.00050,0.00128,0.00159,0.00041,0.84192,0.00208,0.00012\n915,0.00009,0.00064,0.00006,0.00038,0.00001,0.21577,0.78305,0.00000,0.00000\n916,0.11035,0.06197,0.01284,0.05109,0.01952,0.02097,0.70369,0.01076,0.00881\n917,0.00002,0.24837,0.00003,0.00014,0.00043,0.00019,0.74903,0.00179,0.00000\n918,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n919,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n920,0.01190,0.12451,0.00512,0.02131,0.01693,0.02172,0.78945,0.00858,0.00048\n921,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n922,0.70376,0.00854,0.00000,0.00005,0.00001,0.00000,0.28760,0.00002,0.00002\n923,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n924,0.00427,0.27752,0.00871,0.00640,0.01863,0.00409,0.66808,0.01137,0.00093\n925,0.65118,0.02774,0.00005,0.32040,0.00003,0.00017,0.00037,0.00002,0.00004\n926,0.00198,0.39416,0.00068,0.49047,0.00059,0.01519,0.09677,0.00017,0.00000\n927,0.00032,0.14437,0.00026,0.01602,0.00180,0.52617,0.30992,0.00114,0.00000\n928,0.04360,0.12955,0.01245,0.04152,0.04367,0.02855,0.66756,0.02163,0.01147\n929,0.36082,0.03450,0.00047,0.01485,0.00334,0.00609,0.56415,0.01377,0.00201\n930,0.04814,0.36194,0.01410,0.04451,0.04127,0.04136,0.42762,0.01124,0.00982\n931,0.02545,0.32856,0.00014,0.00092,0.00038,0.29481,0.34802,0.00173,0.00000\n932,0.22074,0.03558,0.00007,0.73880,0.00008,0.00011,0.00289,0.00146,0.00026\n933,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n934,0.80620,0.00084,0.00074,0.18406,0.00477,0.00183,0.00126,0.00017,0.00013\n935,0.52751,0.00084,0.00224,0.45740,0.00749,0.00246,0.00174,0.00018,0.00014\n936,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n937,0.00253,0.00257,0.00004,0.59635,0.00002,0.00137,0.39710,0.00002,0.00000\n938,0.66239,0.11427,0.00001,0.00026,0.00009,0.00006,0.21964,0.00318,0.00011\n939,0.00008,0.38401,0.00067,0.00162,0.00423,0.00072,0.60800,0.00067,0.00000\n940,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n941,0.00002,0.20885,0.00001,0.00003,0.00019,0.00003,0.78958,0.00129,0.00000\n942,0.00007,0.86499,0.00001,0.00047,0.00042,0.00015,0.13359,0.00030,0.00000\n943,0.14871,0.00031,0.01925,0.36296,0.43512,0.03354,0.00011,0.00000,0.00001\n944,0.42238,0.03470,0.00634,0.03853,0.01477,0.02214,0.45682,0.00396,0.00036\n945,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n946,0.00417,0.19796,0.02483,0.07801,0.05873,0.01398,0.61385,0.00830,0.00018\n947,0.00014,0.28027,0.00009,0.00008,0.00596,0.00036,0.58786,0.12400,0.00124\n948,0.05273,0.14888,0.01612,0.14733,0.04337,0.02581,0.54075,0.01344,0.01157\n949,0.20852,0.57038,0.00004,0.20432,0.00018,0.00018,0.01626,0.00011,0.00001\n950,0.24922,0.02643,0.00000,0.00005,0.00001,0.00001,0.72426,0.00002,0.00000\n951,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n952,0.21999,0.01724,0.00359,0.71136,0.00338,0.00462,0.03795,0.00125,0.00062\n953,0.10947,0.17092,0.01310,0.03469,0.02930,0.02936,0.58587,0.01519,0.01210\n954,0.01336,0.01186,0.00190,0.21290,0.00270,0.01187,0.74383,0.00146,0.00012\n955,0.59387,0.00090,0.00226,0.38696,0.01064,0.00348,0.00154,0.00019,0.00016\n956,0.06823,0.09206,0.40609,0.07884,0.08142,0.01853,0.23768,0.00866,0.00849\n957,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n958,0.00029,0.20588,0.00750,0.00119,0.02786,0.00349,0.75197,0.00181,0.00001\n959,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n960,0.62724,0.00476,0.00001,0.02470,0.00004,0.00017,0.34233,0.00039,0.00036\n961,0.10558,0.13859,0.03854,0.07821,0.06439,0.08175,0.48706,0.00551,0.00038\n962,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n963,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n964,0.63303,0.13020,0.00442,0.05712,0.02147,0.02093,0.12661,0.00580,0.00042\n965,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n966,0.00090,0.28289,0.00040,0.00043,0.00262,0.00231,0.70837,0.00202,0.00006\n967,0.30436,0.00304,0.00015,0.69075,0.00046,0.00023,0.00097,0.00003,0.00001\n968,0.00119,0.70169,0.00002,0.00133,0.00096,0.00014,0.27888,0.01574,0.00005\n969,0.94496,0.00221,0.00000,0.04833,0.00000,0.00001,0.00421,0.00014,0.00014\n970,0.11244,0.21410,0.00001,0.00005,0.00014,0.00005,0.67242,0.00079,0.00000\n971,0.82590,0.02832,0.00726,0.06955,0.01551,0.01128,0.02887,0.00759,0.00572\n972,0.68853,0.03678,0.01273,0.16300,0.02002,0.01811,0.03964,0.00984,0.01135\n973,0.46955,0.39852,0.00001,0.03807,0.00040,0.00026,0.02686,0.06587,0.00047\n974,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n975,0.00066,0.00040,0.00001,0.33245,0.00000,0.00000,0.66648,0.00000,0.00000\n976,0.63008,0.00441,0.00000,0.02012,0.00002,0.00001,0.34449,0.00057,0.00030\n977,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n978,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n979,0.01382,0.20309,0.00273,0.02897,0.01382,0.00459,0.72885,0.00389,0.00024\n980,0.00674,0.95974,0.00000,0.00470,0.00006,0.00005,0.02185,0.00678,0.00008\n981,0.00008,0.00516,0.00007,0.00028,0.00021,0.42714,0.56689,0.00017,0.00000\n982,0.56756,0.05815,0.00002,0.37369,0.00000,0.00002,0.00048,0.00001,0.00007\n983,0.00055,0.34936,0.00000,0.00014,0.00005,0.00022,0.11107,0.53791,0.00070\n984,0.02014,0.63020,0.00942,0.02367,0.02252,0.02904,0.24806,0.00875,0.00820\n985,0.97137,0.01867,0.00000,0.00129,0.00003,0.00739,0.00046,0.00069,0.00010\n986,0.39910,0.13265,0.00149,0.05070,0.03625,0.00073,0.37437,0.00463,0.00007\n'), index_col='ID')
df_sub2 = pd.read_csv(StringIO('ID,class1,class2,class3,class4,class5,class6,class7,class8,class9\n1,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n2,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n3,0.00054,0.62764,0.00000,0.00007,0.00036,0.00002,0.36996,0.00140,0.00001\n4,0.51136,0.01241,0.00001,0.00031,0.00001,0.00008,0.45745,0.01667,0.00170\n5,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n6,0.94722,0.00039,0.00000,0.00231,0.00001,0.00001,0.05002,0.00001,0.00003\n7,0.00006,0.23009,0.00014,0.00932,0.00096,0.00203,0.75477,0.00262,0.00001\n8,0.48088,0.12269,0.00146,0.11853,0.18199,0.08770,0.00624,0.00046,0.00004\n9,0.26203,0.00360,0.00007,0.73362,0.00011,0.00005,0.00050,0.00002,0.00000\n10,0.00012,0.02432,0.17858,0.00015,0.07120,0.00007,0.72547,0.00009,0.00000\n11,0.00263,0.10553,0.00017,0.71471,0.00770,0.01141,0.15760,0.00025,0.00000\n12,0.01192,0.81433,0.00757,0.01338,0.01149,0.00935,0.12318,0.00455,0.00423\n13,0.92368,0.00028,0.00011,0.01262,0.05290,0.00730,0.00060,0.00101,0.00150\n14,0.00001,0.03054,0.01667,0.00003,0.00708,0.00004,0.94558,0.00005,0.00000\n15,0.00712,0.40879,0.00136,0.02545,0.01285,0.01306,0.52775,0.00360,0.00002\n16,0.47762,0.17516,0.00401,0.11009,0.07644,0.09121,0.03757,0.02765,0.00025\n17,0.01638,0.08538,0.00018,0.00122,0.00037,0.00426,0.88934,0.00285,0.00002\n18,0.05064,0.40840,0.00001,0.00233,0.00013,0.00616,0.49240,0.03964,0.00030\n19,0.01866,0.07628,0.00574,0.02616,0.07020,0.00072,0.79870,0.00326,0.00028\n20,0.02768,0.46824,0.00025,0.00502,0.00405,0.02060,0.46359,0.00687,0.00370\n21,0.00005,0.09113,0.00021,0.00033,0.00089,0.00019,0.90715,0.00005,0.00000\n22,0.04671,0.00444,0.00000,0.14726,0.00001,0.00000,0.00281,0.01411,0.78466\n23,0.31180,0.01266,0.00000,0.00009,0.00002,0.00001,0.67540,0.00002,0.00000\n24,0.00005,0.59990,0.00001,0.00031,0.00076,0.00007,0.38963,0.00925,0.00002\n25,0.58033,0.00994,0.00000,0.00757,0.00001,0.00018,0.39967,0.00168,0.00062\n26,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n27,0.00008,0.85270,0.00000,0.04598,0.00001,0.01712,0.08405,0.00006,0.00000\n28,0.01284,0.22607,0.00009,0.68292,0.00035,0.00003,0.07707,0.00063,0.00000\n29,0.03019,0.18364,0.00030,0.76393,0.00013,0.00150,0.01937,0.00091,0.00003\n30,0.00001,0.87695,0.00080,0.00009,0.00007,0.03922,0.08285,0.00001,0.00000\n31,0.19935,0.05290,0.00527,0.01633,0.03001,0.08964,0.60630,0.00020,0.00000\n32,0.48631,0.01134,0.00003,0.50125,0.00000,0.00001,0.00099,0.00002,0.00005\n33,0.46234,0.01826,0.00047,0.51237,0.00141,0.00048,0.00445,0.00020,0.00002\n34,0.05388,0.11724,0.01436,0.05913,0.02594,0.03208,0.67809,0.01094,0.00834\n35,0.38224,0.45890,0.00001,0.03083,0.00002,0.00001,0.06575,0.00104,0.06120\n36,0.60460,0.00024,0.00002,0.39499,0.00001,0.00001,0.00011,0.00001,0.00001\n37,0.74590,0.02584,0.00012,0.07887,0.00396,0.14205,0.00301,0.00024,0.00001\n38,0.09586,0.46772,0.00575,0.04049,0.00921,0.02704,0.31954,0.01592,0.01848\n39,0.00053,0.05236,0.00002,0.00223,0.00010,0.00003,0.94459,0.00014,0.00000\n40,0.00606,0.09358,0.00017,0.00021,0.00054,0.00009,0.89916,0.00019,0.00000\n41,0.00001,0.07501,0.00001,0.00000,0.00007,0.00004,0.92468,0.00018,0.00000\n42,0.01527,0.82243,0.00023,0.00050,0.00487,0.00013,0.15529,0.00126,0.00002\n43,0.48757,0.35279,0.00001,0.04460,0.00051,0.00027,0.03276,0.08068,0.00081\n44,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n45,0.12150,0.00300,0.00011,0.87427,0.00056,0.00021,0.00026,0.00004,0.00005\n46,0.00048,0.03234,0.00008,0.00017,0.00030,0.64110,0.32514,0.00039,0.00000\n47,0.04360,0.12955,0.01245,0.04152,0.04367,0.02855,0.66756,0.02163,0.01147\n48,0.22959,0.00126,0.00114,0.74614,0.00017,0.00029,0.02140,0.00000,0.00001\n49,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n50,0.00258,0.45446,0.00021,0.00206,0.00255,0.01088,0.51725,0.00998,0.00003\n51,0.01883,0.04397,0.00011,0.46128,0.00143,0.00867,0.46459,0.00103,0.00009\n52,0.55014,0.00015,0.00193,0.05907,0.38416,0.00426,0.00027,0.00000,0.00002\n53,0.00141,0.20030,0.00005,0.31819,0.00009,0.00008,0.47943,0.00045,0.00000\n54,0.01388,0.31785,0.00016,0.02937,0.00153,0.02447,0.61077,0.00196,0.00001\n55,0.16773,0.01819,0.00001,0.00011,0.00002,0.00001,0.81078,0.00304,0.00011\n56,0.00075,0.11836,0.00004,0.00009,0.00093,0.00047,0.87833,0.00103,0.00000\n57,0.00408,0.00002,0.06411,0.93063,0.00042,0.00048,0.00026,0.00000,0.00000\n58,0.21983,0.11806,0.00004,0.65849,0.00002,0.00009,0.00229,0.00113,0.00005\n59,0.00724,0.47233,0.05849,0.42549,0.00083,0.00168,0.03325,0.00068,0.00000\n60,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n61,0.01626,0.49390,0.00009,0.00648,0.00274,0.34446,0.13168,0.00439,0.00001\n62,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n63,0.48433,0.02117,0.00001,0.01404,0.00007,0.00021,0.47928,0.00084,0.00004\n64,0.48719,0.00731,0.00053,0.48527,0.00072,0.00017,0.01869,0.00011,0.00002\n65,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n66,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n67,0.13519,0.13437,0.01867,0.08838,0.02817,0.03037,0.53601,0.01626,0.01258\n68,0.26574,0.24199,0.00000,0.01898,0.00023,0.00024,0.03137,0.43883,0.00261\n69,0.03379,0.20837,0.00016,0.34697,0.00030,0.00008,0.41007,0.00026,0.00000\n70,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n71,0.47580,0.00407,0.00001,0.00027,0.00001,0.00004,0.51977,0.00002,0.00001\n72,0.90722,0.05335,0.00001,0.00738,0.00179,0.00135,0.02156,0.00633,0.00101\n73,0.21672,0.38534,0.00017,0.02297,0.00219,0.00881,0.33018,0.03340,0.00023\n74,0.00001,0.13700,0.00004,0.00005,0.00020,0.00004,0.86225,0.00041,0.00000\n75,0.46046,0.00499,0.44508,0.06077,0.01185,0.00030,0.01654,0.00001,0.00000\n76,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n77,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n78,0.26027,0.00380,0.03844,0.68320,0.00869,0.00275,0.00263,0.00018,0.00004\n79,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n80,0.02012,0.01120,0.00563,0.11958,0.09305,0.72862,0.02134,0.00045,0.00001\n81,0.00030,0.48872,0.00003,0.47542,0.00001,0.00001,0.03551,0.00000,0.00000\n82,0.08819,0.01646,0.04771,0.11205,0.68714,0.01855,0.01786,0.00591,0.00613\n83,0.03609,0.01402,0.00067,0.24296,0.66523,0.02182,0.01909,0.00012,0.00000\n84,0.01306,0.03414,0.00607,0.02081,0.00983,0.00878,0.89745,0.00478,0.00508\n85,0.51813,0.12879,0.00057,0.00640,0.00569,0.00069,0.33241,0.00719,0.00013\n86,0.41745,0.52191,0.00001,0.06000,0.00005,0.00040,0.00017,0.00000,0.00001\n87,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n88,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n89,0.00981,0.02708,0.00129,0.00266,0.01797,0.00005,0.94067,0.00045,0.00002\n90,0.34613,0.61104,0.00001,0.03118,0.00004,0.00826,0.00291,0.00042,0.00001\n91,0.53185,0.00019,0.00003,0.01350,0.00248,0.45183,0.00011,0.00000,0.00001\n92,0.43872,0.01245,0.00882,0.03596,0.44518,0.03135,0.01405,0.00687,0.00659\n93,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n94,0.00072,0.51413,0.00003,0.00010,0.00008,0.00005,0.48471,0.00018,0.00000\n95,0.00004,0.23998,0.00002,0.00025,0.00031,0.00008,0.75892,0.00040,0.00000\n96,0.54416,0.00749,0.00011,0.01871,0.42787,0.00056,0.00104,0.00005,0.00001\n97,0.02838,0.74716,0.00014,0.22006,0.00003,0.00038,0.00385,0.00000,0.00000\n98,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n99,0.00003,0.06638,0.24142,0.00046,0.01001,0.00415,0.67751,0.00004,0.00000\n100,0.00006,0.37734,0.00001,0.00021,0.00022,0.00004,0.62040,0.00171,0.00001\n101,0.99957,0.00010,0.00000,0.00012,0.00002,0.00004,0.00013,0.00001,0.00001\n102,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n103,0.57924,0.05614,0.01761,0.20849,0.02024,0.02968,0.06151,0.01273,0.01436\n104,0.48277,0.37235,0.00001,0.04323,0.00044,0.00022,0.02961,0.07085,0.00053\n105,0.58267,0.01845,0.01058,0.30942,0.01682,0.01309,0.03258,0.00759,0.00880\n106,0.00310,0.39374,0.00042,0.00139,0.00712,0.02512,0.56467,0.00437,0.00007\n107,0.02651,0.01721,0.05145,0.01265,0.65764,0.03466,0.19965,0.00021,0.00002\n108,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n109,0.43000,0.04706,0.00252,0.04091,0.00393,0.00108,0.47433,0.00017,0.00000\n110,0.04007,0.02556,0.01363,0.08973,0.44302,0.11105,0.25653,0.01072,0.00969\n111,0.18452,0.06097,0.00010,0.04831,0.00116,0.04588,0.65842,0.00063,0.00001\n112,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n113,0.97515,0.01689,0.00000,0.00669,0.00001,0.00016,0.00057,0.00051,0.00002\n114,0.94537,0.00403,0.00001,0.04464,0.00003,0.00002,0.00374,0.00199,0.00017\n115,0.00998,0.02381,0.00458,0.01117,0.00762,0.00901,0.92721,0.00351,0.00311\n116,0.64612,0.00061,0.00004,0.35290,0.00001,0.00003,0.00017,0.00004,0.00008\n117,0.00004,0.27447,0.00192,0.00075,0.00136,0.00934,0.71161,0.00051,0.00000\n118,0.75053,0.00067,0.00005,0.24848,0.00007,0.00005,0.00014,0.00000,0.00001\n119,0.00033,0.00671,0.00000,0.00002,0.00001,0.00000,0.90787,0.00471,0.08035\n120,0.26408,0.07664,0.00017,0.04966,0.00342,0.00206,0.59684,0.00576,0.00137\n121,0.01999,0.01166,0.00791,0.00973,0.00153,0.93029,0.01729,0.00160,0.00000\n122,0.00004,0.20400,0.00004,0.00037,0.00093,0.00016,0.79411,0.00035,0.00000\n123,0.00002,0.05817,0.33902,0.00035,0.00667,0.00874,0.58701,0.00002,0.00000\n124,0.06359,0.14900,0.01678,0.12233,0.02675,0.04031,0.55528,0.01324,0.01272\n125,0.01000,0.40509,0.00021,0.16679,0.01873,0.26266,0.12589,0.01062,0.00000\n126,0.00159,0.85580,0.00033,0.01138,0.01187,0.03987,0.07824,0.00092,0.00000\n127,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n128,0.91978,0.00634,0.00006,0.04241,0.00073,0.02274,0.00651,0.00121,0.00022\n129,0.56265,0.10158,0.01275,0.12208,0.02968,0.02423,0.08906,0.02461,0.03336\n130,0.00018,0.00801,0.00002,0.00072,0.00034,0.97976,0.01009,0.00088,0.00000\n131,0.01777,0.06318,0.01031,0.05330,0.01417,0.01146,0.81638,0.00741,0.00602\n132,0.53058,0.33658,0.00001,0.03310,0.00028,0.00019,0.02861,0.07003,0.00062\n133,0.00004,0.22210,0.00016,0.00028,0.00118,0.00116,0.77258,0.00250,0.00000\n134,0.39948,0.06289,0.01728,0.29425,0.03995,0.03405,0.11047,0.02083,0.02080\n135,0.50756,0.03274,0.00006,0.00218,0.00021,0.01730,0.43985,0.00009,0.00001\n136,0.54668,0.27920,0.00003,0.04320,0.00123,0.08425,0.04353,0.00188,0.00000\n137,0.15594,0.71990,0.00000,0.00040,0.00012,0.00008,0.09542,0.02792,0.00022\n138,0.00027,0.62602,0.00013,0.00118,0.01014,0.03039,0.31061,0.02115,0.00011\n139,0.00763,0.00005,0.05868,0.93134,0.00100,0.00089,0.00041,0.00000,0.00000\n140,0.00177,0.00532,0.00002,0.84565,0.00001,0.00001,0.14721,0.00001,0.00000\n141,0.00038,0.04392,0.02698,0.00021,0.06454,0.00010,0.86294,0.00092,0.00001\n142,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n143,0.00004,0.00022,0.00000,0.00001,0.00000,0.00000,0.99973,0.00000,0.00000\n144,0.16165,0.20732,0.00006,0.62606,0.00002,0.00026,0.00208,0.00253,0.00002\n145,0.12683,0.04206,0.01538,0.65931,0.03152,0.01862,0.08293,0.01007,0.01328\n146,0.01466,0.06674,0.00852,0.01570,0.01237,0.01084,0.85101,0.01294,0.00722\n147,0.07064,0.00008,0.00004,0.92914,0.00002,0.00001,0.00007,0.00000,0.00000\n148,0.05812,0.02360,0.00007,0.91360,0.00001,0.00001,0.00367,0.00083,0.00009\n149,0.00150,0.16144,0.00072,0.00053,0.60906,0.00226,0.22403,0.00046,0.00000\n150,0.48761,0.00872,0.00161,0.46095,0.00152,0.00043,0.03894,0.00016,0.00007\n151,0.03265,0.11458,0.00080,0.01577,0.76324,0.00204,0.07036,0.00056,0.00000\n152,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n153,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n154,0.19307,0.00026,0.00009,0.80629,0.00007,0.00002,0.00020,0.00000,0.00000\n155,0.00407,0.98056,0.00001,0.00433,0.00003,0.00803,0.00193,0.00103,0.00001\n156,0.00188,0.48839,0.00055,0.00187,0.01034,0.01399,0.47828,0.00467,0.00003\n157,0.11785,0.03939,0.00018,0.00977,0.00038,0.00025,0.83192,0.00025,0.00001\n158,0.00001,0.05020,0.00001,0.00004,0.00002,0.00004,0.94966,0.00002,0.00000\n159,0.00011,0.03991,0.00047,0.00084,0.00078,0.01042,0.94647,0.00095,0.00005\n160,0.79546,0.11556,0.00011,0.06858,0.00015,0.00023,0.01956,0.00026,0.00009\n161,0.48448,0.37672,0.00001,0.04388,0.00045,0.00024,0.03191,0.06180,0.00051\n162,0.02710,0.00383,0.00001,0.00056,0.00003,0.84221,0.12624,0.00002,0.00000\n163,0.00523,0.00588,0.00005,0.60739,0.00040,0.00003,0.01801,0.14839,0.21462\n164,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n165,0.47066,0.37654,0.00001,0.04286,0.00054,0.00059,0.02796,0.08036,0.00049\n166,0.23367,0.72949,0.00000,0.01444,0.00001,0.00203,0.02034,0.00001,0.00001\n167,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n168,0.06955,0.20498,0.01439,0.08209,0.02169,0.02375,0.55927,0.01351,0.01077\n169,0.86294,0.00010,0.00002,0.13651,0.00011,0.00004,0.00004,0.00000,0.00024\n170,0.02465,0.04875,0.00125,0.31631,0.00050,0.08589,0.52182,0.00082,0.00001\n171,0.00026,0.65269,0.00001,0.00873,0.00041,0.00031,0.33686,0.00073,0.00000\n172,0.00541,0.02086,0.00008,0.96606,0.00004,0.00004,0.00747,0.00004,0.00000\n173,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n174,0.66049,0.02553,0.01459,0.16267,0.04101,0.03616,0.04148,0.00930,0.00877\n175,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n176,0.05361,0.04504,0.00288,0.00616,0.65249,0.00028,0.23766,0.00156,0.00032\n177,0.97518,0.00018,0.00000,0.01678,0.00001,0.00057,0.00013,0.00002,0.00713\n178,0.29884,0.00012,0.00309,0.51475,0.15157,0.03154,0.00008,0.00000,0.00001\n179,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n180,0.00002,0.42508,0.00000,0.00007,0.00005,0.00002,0.57463,0.00013,0.00000\n181,0.00008,0.52705,0.00000,0.00003,0.00003,0.00002,0.47271,0.00008,0.00000\n182,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n183,0.00021,0.41907,0.00003,0.00005,0.00003,0.00001,0.58052,0.00008,0.00000\n184,0.37709,0.00231,0.00174,0.04360,0.00556,0.56894,0.00069,0.00006,0.00001\n185,0.01966,0.00526,0.00004,0.60429,0.00001,0.00002,0.37072,0.00000,0.00000\n186,0.08422,0.00011,0.00005,0.91547,0.00004,0.00001,0.00010,0.00000,0.00000\n187,0.10947,0.17092,0.01310,0.03469,0.02930,0.02936,0.58587,0.01519,0.01210\n188,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n189,0.00005,0.37842,0.00001,0.00005,0.00023,0.00004,0.61734,0.00383,0.00003\n190,0.03211,0.19859,0.01299,0.04127,0.01726,0.01521,0.66768,0.00865,0.00624\n191,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n192,0.00004,0.29187,0.00013,0.00022,0.00148,0.00356,0.70066,0.00204,0.00000\n193,0.00539,0.03889,0.00077,0.72578,0.00110,0.00021,0.22782,0.00004,0.00000\n194,0.16905,0.00001,0.00880,0.16992,0.63972,0.01248,0.00002,0.00000,0.00000\n195,0.23861,0.24518,0.00504,0.02540,0.04363,0.00046,0.40241,0.03901,0.00027\n196,0.00339,0.00001,0.05466,0.94139,0.00021,0.00016,0.00018,0.00000,0.00000\n197,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n198,0.07338,0.22251,0.01272,0.06735,0.01885,0.55967,0.02845,0.00773,0.00934\n199,0.53461,0.02791,0.00000,0.00012,0.00000,0.00000,0.43722,0.00004,0.00010\n200,0.45036,0.23941,0.00013,0.00567,0.00326,0.21277,0.08458,0.00381,0.00000\n201,0.52895,0.33999,0.00001,0.03194,0.00036,0.00025,0.02557,0.07233,0.00060\n202,0.45074,0.00020,0.00021,0.00840,0.19877,0.34164,0.00004,0.00000,0.00000\n203,0.00001,0.50711,0.00002,0.00010,0.00004,0.00387,0.48828,0.00057,0.00000\n204,0.00001,0.28476,0.00000,0.00001,0.00002,0.00001,0.71396,0.00123,0.00000\n205,0.31730,0.00013,0.00532,0.04917,0.61522,0.01255,0.00026,0.00001,0.00004\n206,0.99951,0.00003,0.00000,0.00043,0.00000,0.00001,0.00001,0.00000,0.00001\n207,0.31286,0.07774,0.00733,0.10946,0.03577,0.07833,0.37699,0.00146,0.00006\n208,0.06828,0.25526,0.01635,0.07233,0.05329,0.02822,0.48605,0.01131,0.00890\n209,0.08516,0.00273,0.02250,0.07745,0.61123,0.19965,0.00099,0.00026,0.00003\n210,0.43726,0.01835,0.00047,0.00083,0.01188,0.00082,0.53015,0.00008,0.00016\n211,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n212,0.21847,0.00043,0.00041,0.77943,0.00075,0.00013,0.00038,0.00000,0.00000\n213,0.68923,0.02170,0.00000,0.02764,0.00001,0.00001,0.26096,0.00040,0.00005\n214,0.01792,0.61713,0.00042,0.05377,0.01664,0.27065,0.02175,0.00171,0.00001\n215,0.01321,0.08604,0.00788,0.01416,0.00975,0.00910,0.84923,0.00554,0.00509\n216,0.03986,0.03262,0.01106,0.03643,0.73022,0.01360,0.12050,0.00926,0.00645\n217,0.01192,0.81433,0.00757,0.01338,0.01149,0.00935,0.12318,0.00455,0.00423\n218,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n219,0.06012,0.00280,0.00542,0.09467,0.19148,0.00549,0.64001,0.00000,0.00000\n220,0.00027,0.03594,0.00703,0.00257,0.00148,0.34593,0.60666,0.00012,0.00000\n221,0.00000,0.03540,0.00001,0.00003,0.00002,0.00002,0.96450,0.00002,0.00000\n222,0.18692,0.77637,0.00000,0.00012,0.00093,0.00095,0.03412,0.00056,0.00003\n223,0.00376,0.00005,0.06238,0.93258,0.00036,0.00052,0.00035,0.00000,0.00000\n224,0.51939,0.34760,0.00001,0.03128,0.00038,0.00022,0.02827,0.07235,0.00050\n225,0.24867,0.68682,0.00002,0.01444,0.00083,0.00075,0.03342,0.01461,0.00044\n226,0.19508,0.01870,0.00018,0.54244,0.00063,0.00155,0.24137,0.00002,0.00003\n227,0.55397,0.02713,0.00003,0.01279,0.00004,0.00012,0.40589,0.00003,0.00000\n228,0.25937,0.18990,0.01100,0.02770,0.07778,0.00043,0.40299,0.03051,0.00032\n229,0.49798,0.35370,0.00003,0.04962,0.00039,0.00020,0.02681,0.07065,0.00063\n230,0.08236,0.00823,0.00015,0.36874,0.00075,0.00003,0.53907,0.00061,0.00006\n231,0.00092,0.00197,0.00005,0.46319,0.00001,0.00000,0.53385,0.00001,0.00000\n232,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n233,0.04288,0.00505,0.00002,0.29511,0.00003,0.00002,0.65684,0.00005,0.00000\n234,0.25733,0.01054,0.00001,0.00774,0.00002,0.00015,0.72409,0.00011,0.00001\n235,0.92985,0.00131,0.00021,0.00307,0.06223,0.00046,0.00280,0.00002,0.00005\n236,0.48878,0.20036,0.01411,0.07501,0.10099,0.01769,0.08485,0.00938,0.00883\n237,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n238,0.05761,0.24385,0.00889,0.04510,0.01541,0.01880,0.59228,0.00798,0.01008\n239,0.52502,0.00699,0.00020,0.46366,0.00077,0.00036,0.00273,0.00018,0.00009\n240,0.83017,0.00214,0.00003,0.16635,0.00020,0.00010,0.00084,0.00005,0.00012\n241,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n242,0.00009,0.99379,0.00000,0.00003,0.00004,0.00092,0.00492,0.00021,0.00000\n243,0.00326,0.00552,0.00048,0.66822,0.00483,0.00010,0.31756,0.00003,0.00000\n244,0.51510,0.03217,0.00016,0.45198,0.00003,0.00008,0.00021,0.00019,0.00008\n245,0.00000,0.09479,0.00001,0.00001,0.00004,0.00001,0.90502,0.00012,0.00000\n246,0.01616,0.47371,0.00001,0.50210,0.00001,0.00000,0.00799,0.00002,0.00000\n247,0.22816,0.16144,0.01794,0.15272,0.01245,0.04781,0.35839,0.02090,0.00020\n248,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n249,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n250,0.75966,0.01735,0.00003,0.07343,0.00036,0.00045,0.14867,0.00002,0.00003\n251,0.17504,0.41641,0.00019,0.02041,0.00222,0.01080,0.34397,0.03067,0.00029\n252,0.03618,0.11343,0.00693,0.09514,0.02306,0.12261,0.60256,0.00009,0.00000\n253,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n254,0.27980,0.00477,0.00002,0.08944,0.00016,0.00020,0.62552,0.00007,0.00002\n255,0.51168,0.14887,0.02116,0.13086,0.02056,0.03417,0.09059,0.02518,0.01693\n256,0.22643,0.02052,0.01101,0.12731,0.01887,0.55467,0.02650,0.00740,0.00729\n257,0.10207,0.55462,0.00104,0.21082,0.04535,0.00173,0.08437,0.00000,0.00000\n258,0.00775,0.55938,0.00039,0.00304,0.00576,0.01172,0.37713,0.01784,0.01699\n259,0.01187,0.00032,0.00067,0.97986,0.00041,0.00680,0.00007,0.00000,0.00000\n260,0.23254,0.54706,0.00019,0.19058,0.00090,0.00374,0.02492,0.00005,0.00002\n261,0.24068,0.02794,0.00034,0.71628,0.00069,0.00023,0.01326,0.00045,0.00013\n262,0.01839,0.03554,0.00006,0.58846,0.00006,0.00006,0.35624,0.00116,0.00003\n263,0.00006,0.85314,0.00658,0.00369,0.01639,0.02485,0.09498,0.00031,0.00000\n264,0.24267,0.03672,0.01266,0.04603,0.58444,0.03404,0.02420,0.00961,0.00963\n265,0.50868,0.01469,0.01106,0.03641,0.37903,0.01794,0.01733,0.00741,0.00745\n266,0.00006,0.27637,0.00010,0.00031,0.00050,0.00059,0.72053,0.00154,0.00000\n267,0.00000,0.05286,0.00001,0.00002,0.00013,0.00002,0.94298,0.00393,0.00005\n268,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n269,0.00401,0.39222,0.00009,0.00502,0.00025,0.00011,0.59795,0.00035,0.00000\n270,0.00078,0.31606,0.00049,0.00178,0.00564,0.27313,0.39444,0.00767,0.00000\n271,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n272,0.03100,0.00189,0.00002,0.00003,0.00000,0.04861,0.91843,0.00002,0.00000\n273,0.24834,0.00277,0.00028,0.23157,0.51349,0.00092,0.00255,0.00006,0.00002\n274,0.61928,0.35195,0.00000,0.00095,0.00002,0.00016,0.02184,0.00260,0.00320\n275,0.00477,0.00087,0.00005,0.41829,0.00000,0.00000,0.57602,0.00000,0.00000\n276,0.22839,0.01328,0.00003,0.61301,0.00007,0.00009,0.14507,0.00006,0.00000\n277,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n278,0.61984,0.12841,0.01306,0.09516,0.01809,0.02305,0.06739,0.02115,0.01384\n279,0.00018,0.01724,0.00000,0.00010,0.00004,0.00000,0.00945,0.05945,0.91354\n280,0.01466,0.57831,0.00001,0.00307,0.00005,0.00211,0.40158,0.00021,0.00000\n281,0.38325,0.06839,0.00364,0.17971,0.16641,0.16414,0.03191,0.00251,0.00005\n282,0.05089,0.01792,0.00027,0.00044,0.00423,0.00006,0.92615,0.00004,0.00000\n283,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n284,0.56777,0.00319,0.00001,0.04428,0.00000,0.00001,0.38474,0.00000,0.00000\n285,0.23089,0.10229,0.02216,0.36970,0.06319,0.06137,0.12004,0.01514,0.01521\n286,0.57850,0.00201,0.00003,0.41730,0.00004,0.00001,0.00071,0.00033,0.00107\n287,0.02490,0.25871,0.01238,0.02628,0.03761,0.02248,0.56690,0.01479,0.03595\n288,0.18489,0.00143,0.00007,0.81306,0.00006,0.00004,0.00040,0.00001,0.00004\n289,0.01532,0.28471,0.00010,0.00863,0.00162,0.01296,0.67267,0.00394,0.00005\n290,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n291,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000\n292,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n293,0.52675,0.00226,0.00000,0.00046,0.00006,0.06770,0.38448,0.01818,0.00011\n294,0.04374,0.10138,0.01175,0.10645,0.01805,0.01607,0.68716,0.00813,0.00727\n295,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n296,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n297,0.67393,0.00077,0.00006,0.32279,0.00151,0.00015,0.00074,0.00001,0.00004\n298,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n299,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n300,0.00062,0.18718,0.00000,0.00036,0.00181,0.00043,0.13184,0.64594,0.03182\n301,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n302,0.01363,0.00774,0.00008,0.00033,0.00039,0.51320,0.46457,0.00006,0.00000\n303,0.00013,0.06642,0.00061,0.00056,0.00034,0.05178,0.36777,0.00743,0.50496\n304,0.00004,0.00043,0.00007,0.00265,0.00011,0.51371,0.48299,0.00000,0.00000\n305,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n306,0.00038,0.59542,0.04222,0.03033,0.00690,0.00120,0.32052,0.00302,0.00001\n307,0.71348,0.02212,0.00908,0.13688,0.01647,0.01352,0.07358,0.00745,0.00743\n308,0.50049,0.00363,0.00012,0.01368,0.09741,0.38339,0.00111,0.00017,0.00000\n309,0.00798,0.17982,0.00016,0.00396,0.00542,0.76050,0.03903,0.00313,0.00000\n310,0.00015,0.74912,0.00000,0.00024,0.00001,0.00004,0.25040,0.00004,0.00000\n311,0.21170,0.02021,0.00840,0.69507,0.01696,0.01266,0.01871,0.00696,0.00933\n312,0.00650,0.00004,0.05850,0.93188,0.00180,0.00095,0.00033,0.00000,0.00000\n313,0.28886,0.06150,0.00000,0.00374,0.00001,0.00039,0.64544,0.00006,0.00000\n314,0.48350,0.00738,0.00008,0.50544,0.00059,0.00022,0.00221,0.00052,0.00006\n315,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n316,0.09532,0.02867,0.01172,0.79194,0.01417,0.01357,0.02912,0.00846,0.00703\n317,0.48253,0.00162,0.00067,0.09746,0.13312,0.28452,0.00006,0.00001,0.00000\n318,0.00007,0.11988,0.00015,0.00026,0.00278,0.00042,0.87545,0.00098,0.00001\n319,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n320,0.33595,0.00063,0.00004,0.61168,0.00000,0.00000,0.05169,0.00000,0.00001\n321,0.27671,0.12324,0.00009,0.01281,0.00072,0.00069,0.58509,0.00064,0.00001\n322,0.42498,0.57488,0.00000,0.00004,0.00000,0.00000,0.00010,0.00000,0.00000\n323,0.07330,0.04781,0.01082,0.07187,0.02120,0.02094,0.73531,0.00976,0.00899\n324,0.00004,0.01240,0.00001,0.00003,0.00002,0.00007,0.61805,0.01758,0.35180\n325,0.02441,0.58863,0.00010,0.38565,0.00001,0.00025,0.00092,0.00003,0.00000\n326,0.00054,0.38102,0.00027,0.00050,0.00218,0.00724,0.60284,0.00524,0.00017\n327,0.46008,0.23836,0.01272,0.05211,0.02835,0.02183,0.15336,0.02216,0.01103\n328,0.76695,0.00100,0.00083,0.21651,0.00437,0.00196,0.00832,0.00001,0.00005\n329,0.00004,0.87770,0.00000,0.00004,0.00004,0.00008,0.12161,0.00049,0.00000\n330,0.28056,0.00065,0.02085,0.02674,0.21208,0.45859,0.00051,0.00002,0.00001\n331,0.50365,0.07576,0.00038,0.14838,0.00098,0.00036,0.27038,0.00004,0.00007\n332,0.10323,0.02981,0.00153,0.79820,0.00138,0.06548,0.00037,0.00000,0.00000\n333,0.54125,0.00009,0.00209,0.07276,0.37977,0.00382,0.00019,0.00000,0.00003\n334,0.00726,0.64286,0.00000,0.00107,0.00044,0.01314,0.33462,0.00061,0.00000\n335,0.00359,0.04441,0.00005,0.45264,0.00762,0.46339,0.02393,0.00436,0.00001\n336,0.10019,0.10422,0.01273,0.02999,0.02824,0.04181,0.65783,0.01464,0.01035\n337,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n338,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n339,0.21877,0.00895,0.00001,0.00005,0.00001,0.00003,0.74299,0.00165,0.02754\n340,0.25396,0.25661,0.00119,0.22288,0.00194,0.00685,0.24150,0.01488,0.00019\n341,0.29701,0.68307,0.00000,0.01203,0.00004,0.00004,0.00746,0.00034,0.00001\n342,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n343,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n344,0.00256,0.04651,0.00000,0.00015,0.00006,0.00050,0.01933,0.92921,0.00168\n345,0.99948,0.00005,0.00000,0.00021,0.00000,0.00000,0.00002,0.00001,0.00023\n346,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n347,0.02527,0.00017,0.00511,0.13773,0.81547,0.01574,0.00050,0.00000,0.00001\n348,0.00001,0.08952,0.00010,0.00002,0.00111,0.00006,0.90745,0.00172,0.00001\n349,0.00025,0.13076,0.00001,0.00054,0.00055,0.00007,0.19984,0.12007,0.54791\n350,0.08971,0.45817,0.00003,0.30079,0.00001,0.00001,0.15126,0.00002,0.00000\n351,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n352,0.67021,0.00716,0.00013,0.31713,0.00008,0.00027,0.00492,0.00008,0.00002\n353,0.00063,0.06307,0.00002,0.93608,0.00000,0.00009,0.00011,0.00000,0.00000\n354,0.36668,0.00010,0.00001,0.63314,0.00000,0.00001,0.00004,0.00000,0.00002\n355,0.00173,0.01968,0.00062,0.03857,0.00203,0.00004,0.93730,0.00003,0.00000\n356,0.54389,0.06966,0.00012,0.00057,0.01358,0.01950,0.35249,0.00017,0.00002\n357,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n358,0.84526,0.01677,0.00002,0.00590,0.00337,0.11138,0.01696,0.00033,0.00001\n359,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n360,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n361,0.00003,0.09982,0.00066,0.00031,0.00052,0.02435,0.87395,0.00036,0.00000\n362,0.44929,0.00569,0.00000,0.00003,0.00000,0.00000,0.54499,0.00000,0.00000\n363,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n364,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000\n365,0.14752,0.41218,0.00776,0.03541,0.00581,0.01059,0.34871,0.03182,0.00020\n366,0.07468,0.61393,0.00003,0.30769,0.00003,0.00001,0.00354,0.00009,0.00000\n367,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n368,0.05452,0.01687,0.01601,0.81008,0.04500,0.02083,0.01576,0.01065,0.01028\n369,0.69178,0.19343,0.00006,0.09575,0.00034,0.00167,0.01229,0.00442,0.00026\n370,0.02031,0.26360,0.01177,0.04076,0.01762,0.01690,0.61395,0.00775,0.00734\n371,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n372,0.74286,0.00041,0.00083,0.14907,0.06020,0.04648,0.00013,0.00001,0.00001\n373,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n374,0.00089,0.51300,0.00000,0.00094,0.00009,0.00002,0.48477,0.00029,0.00000\n375,0.00031,0.03285,0.00003,0.00211,0.00020,0.57018,0.39419,0.00013,0.00000\n376,0.13595,0.00003,0.02640,0.34628,0.43263,0.05865,0.00005,0.00000,0.00000\n377,0.34626,0.00186,0.00054,0.03612,0.03972,0.00034,0.57507,0.00008,0.00001\n378,0.84300,0.02363,0.00019,0.00653,0.00301,0.07514,0.04569,0.00269,0.00012\n379,0.56777,0.00284,0.00000,0.00004,0.00000,0.00000,0.42935,0.00000,0.00000\n380,0.00006,0.21396,0.00018,0.00016,0.00197,0.00017,0.78220,0.00128,0.00002\n381,0.00947,0.02109,0.00003,0.39959,0.00005,0.00002,0.56934,0.00040,0.00001\n382,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n383,0.00004,0.26008,0.00076,0.00122,0.00073,0.00030,0.73650,0.00037,0.00000\n384,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n385,0.24358,0.00008,0.01111,0.33259,0.34944,0.06313,0.00006,0.00000,0.00001\n386,0.00193,0.25224,0.01372,0.04595,0.05323,0.03218,0.59618,0.00455,0.00002\n387,0.00449,0.01430,0.00002,0.29020,0.00015,0.00010,0.69045,0.00029,0.00000\n388,0.03281,0.08856,0.00062,0.01967,0.00203,0.26998,0.57918,0.00715,0.00000\n389,0.72995,0.00059,0.00001,0.20601,0.06192,0.00011,0.00140,0.00000,0.00001\n390,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n391,0.02171,0.40255,0.00590,0.01259,0.54883,0.00749,0.00062,0.00030,0.00001\n392,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n393,0.00020,0.25153,0.00265,0.00098,0.02605,0.00065,0.71608,0.00182,0.00004\n394,0.05084,0.17625,0.01608,0.05192,0.02369,0.02413,0.63649,0.01185,0.00875\n395,0.54257,0.05172,0.01500,0.23314,0.02981,0.02211,0.07951,0.01421,0.01193\n396,0.20261,0.46445,0.00024,0.02174,0.00511,0.00607,0.27020,0.02933,0.00025\n397,0.00004,0.08680,0.00239,0.00010,0.03237,0.00047,0.87313,0.00469,0.00001\n398,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n399,0.89248,0.00261,0.00012,0.00228,0.09793,0.00101,0.00345,0.00006,0.00006\n400,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n401,0.00218,0.37402,0.00047,0.00087,0.00804,0.00701,0.60529,0.00211,0.00001\n402,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n403,0.18379,0.04838,0.02435,0.50888,0.04486,0.02534,0.13578,0.01450,0.01412\n404,0.50466,0.00562,0.00084,0.47807,0.00091,0.00022,0.00957,0.00008,0.00003\n405,0.01033,0.12256,0.00011,0.23781,0.00038,0.00076,0.62722,0.00083,0.00000\n406,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n407,0.00217,0.33628,0.00003,0.00255,0.00044,0.00542,0.08651,0.05473,0.51187\n408,0.00002,0.32325,0.00073,0.00013,0.00012,0.00014,0.67515,0.00046,0.00000\n409,0.00120,0.66028,0.00005,0.00092,0.16775,0.00133,0.15561,0.01281,0.00005\n410,0.00019,0.19557,0.00049,0.00051,0.00107,0.02469,0.77113,0.00631,0.00004\n411,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n412,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n413,0.29255,0.00225,0.07972,0.61200,0.00777,0.00178,0.00390,0.00002,0.00001\n414,0.55665,0.00363,0.00006,0.00903,0.07021,0.35911,0.00120,0.00011,0.00000\n415,0.01384,0.83701,0.00699,0.01170,0.01296,0.01016,0.09868,0.00417,0.00449\n416,0.02326,0.01161,0.01314,0.06047,0.83385,0.03335,0.01260,0.00600,0.00572\n417,0.65916,0.19186,0.00000,0.00038,0.00011,0.00006,0.14810,0.00032,0.00000\n418,0.89097,0.01652,0.00137,0.08131,0.00405,0.00042,0.00441,0.00088,0.00007\n419,0.00001,0.01646,0.00000,0.00000,0.00002,0.00002,0.98286,0.00062,0.00001\n420,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n421,0.81085,0.01920,0.00068,0.03692,0.09507,0.01930,0.01724,0.00059,0.00015\n422,0.54194,0.32643,0.00001,0.03512,0.00034,0.00021,0.03117,0.06421,0.00057\n423,0.24559,0.05554,0.01841,0.47783,0.03489,0.03277,0.11244,0.01124,0.01128\n424,0.00001,0.99801,0.00000,0.00003,0.00001,0.00003,0.00191,0.00000,0.00000\n425,0.02774,0.00969,0.32438,0.01498,0.03062,0.53945,0.05302,0.00012,0.00000\n426,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n427,0.62379,0.05683,0.01359,0.11070,0.01961,0.02625,0.11689,0.01558,0.01676\n428,0.69848,0.02329,0.01482,0.04483,0.03960,0.13605,0.02348,0.00938,0.01006\n429,0.02963,0.00003,0.01362,0.42969,0.50682,0.02009,0.00012,0.00000,0.00000\n430,0.03394,0.04555,0.00810,0.66725,0.13823,0.01682,0.08776,0.00226,0.00009\n431,0.06019,0.29608,0.00055,0.06391,0.01436,0.02980,0.53090,0.00419,0.00002\n432,0.01899,0.07154,0.00742,0.01806,0.02208,0.01477,0.83577,0.00593,0.00544\n433,0.00136,0.28671,0.01664,0.03597,0.00336,0.03461,0.61084,0.01051,0.00000\n434,0.58048,0.05383,0.00004,0.35644,0.00138,0.00065,0.00429,0.00266,0.00023\n435,0.04350,0.29638,0.01246,0.05322,0.03567,0.07728,0.47816,0.00323,0.00009\n436,0.05459,0.40497,0.00000,0.00012,0.00002,0.00002,0.54013,0.00008,0.00007\n437,0.00001,0.02158,0.00015,0.04920,0.00001,0.00362,0.92540,0.00003,0.00000\n438,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n439,0.00081,0.47860,0.00038,0.00455,0.00100,0.01452,0.49999,0.00014,0.00000\n440,0.00012,0.04114,0.00001,0.00004,0.00007,0.00000,0.43400,0.01448,0.51014\n441,0.31273,0.01539,0.00373,0.57422,0.06135,0.00385,0.02867,0.00004,0.00002\n442,0.00002,0.03161,0.00003,0.00004,0.00004,0.00001,0.64682,0.00635,0.31508\n443,0.00956,0.22606,0.00011,0.13714,0.00035,0.00064,0.62498,0.00115,0.00001\n444,0.72503,0.00083,0.09803,0.14206,0.00123,0.00099,0.03182,0.00000,0.00000\n445,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n446,0.57101,0.00035,0.00000,0.03667,0.00000,0.00000,0.39197,0.00000,0.00000\n447,0.21545,0.29779,0.00002,0.47955,0.00005,0.00011,0.00484,0.00167,0.00051\n448,0.21726,0.14221,0.00018,0.58512,0.00038,0.00116,0.03125,0.02214,0.00030\n449,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n450,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n451,0.00791,0.00045,0.00001,0.19064,0.00000,0.00000,0.80099,0.00000,0.00000\n452,0.04408,0.00545,0.00055,0.02446,0.00622,0.00023,0.91845,0.00045,0.00011\n453,0.00439,0.00025,0.00001,0.80170,0.00000,0.00000,0.19365,0.00000,0.00000\n454,0.03988,0.21260,0.01088,0.05062,0.02528,0.02492,0.61549,0.01086,0.00947\n455,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n456,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n457,0.19725,0.04018,0.00006,0.01609,0.00047,0.00013,0.74416,0.00159,0.00007\n458,0.00907,0.22975,0.00007,0.74246,0.00005,0.00001,0.01855,0.00004,0.00000\n459,0.01645,0.34737,0.00188,0.03491,0.04194,0.06394,0.48212,0.01127,0.00011\n460,0.77980,0.00286,0.05669,0.14517,0.00485,0.00067,0.00981,0.00008,0.00007\n461,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n462,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n463,0.01806,0.24057,0.00486,0.07488,0.02393,0.04103,0.59276,0.00389,0.00002\n464,0.00375,0.28780,0.00404,0.06290,0.02172,0.02212,0.59065,0.00676,0.00026\n465,0.46909,0.38566,0.00001,0.04168,0.00044,0.00031,0.02914,0.07328,0.00039\n466,0.58872,0.02306,0.00088,0.20657,0.01866,0.15843,0.00176,0.00180,0.00012\n467,0.66023,0.02068,0.00034,0.01627,0.15378,0.13597,0.01244,0.00018,0.00011\n468,0.23528,0.05798,0.00010,0.00674,0.00446,0.00024,0.69063,0.00432,0.00025\n469,0.38648,0.31154,0.00001,0.03140,0.00036,0.00019,0.21548,0.05390,0.00065\n470,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n471,0.73914,0.06407,0.01187,0.07416,0.02318,0.02373,0.04307,0.01144,0.00934\n472,0.00001,0.07000,0.00003,0.00002,0.00009,0.00002,0.92937,0.00046,0.00000\n473,0.00016,0.36653,0.00013,0.00032,0.00083,0.00013,0.63041,0.00149,0.00000\n474,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n475,0.15814,0.00335,0.03632,0.17632,0.14891,0.01639,0.46055,0.00001,0.00001\n476,0.03988,0.21260,0.01088,0.05062,0.02528,0.02492,0.61549,0.01086,0.00947\n477,0.03394,0.00032,0.00008,0.67573,0.00002,0.00003,0.28988,0.00000,0.00000\n478,0.00617,0.00038,0.53814,0.25494,0.07701,0.12249,0.00087,0.00000,0.00000\n479,0.05987,0.10612,0.00043,0.81999,0.00021,0.00136,0.01167,0.00035,0.00000\n480,0.11959,0.02019,0.00010,0.84413,0.00008,0.00001,0.01584,0.00004,0.00002\n481,0.01944,0.01967,0.00060,0.30049,0.00041,0.00223,0.65709,0.00006,0.00001\n482,0.00012,0.32477,0.00055,0.00051,0.00350,0.00012,0.66906,0.00137,0.00000\n483,0.29536,0.01038,0.00001,0.00012,0.00001,0.00039,0.69368,0.00005,0.00000\n484,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n485,0.00958,0.09055,0.01041,0.00819,0.17386,0.01031,0.69041,0.00653,0.00016\n486,0.22116,0.00086,0.00018,0.77679,0.00052,0.00008,0.00038,0.00001,0.00002\n487,0.00339,0.23086,0.00120,0.00198,0.01629,0.00201,0.74223,0.00203,0.00001\n488,0.39552,0.00266,0.00001,0.03259,0.00000,0.00002,0.56913,0.00003,0.00004\n489,0.07233,0.01945,0.06416,0.23551,0.08624,0.48097,0.01902,0.01145,0.01086\n490,0.04176,0.01603,0.03433,0.13815,0.71843,0.02045,0.01527,0.00825,0.00733\n491,0.00076,0.00084,0.00004,0.29857,0.00000,0.00001,0.69978,0.00000,0.00000\n492,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n493,0.02149,0.26745,0.00009,0.67216,0.00000,0.00002,0.03879,0.00000,0.00000\n494,0.52358,0.37105,0.00002,0.04319,0.00020,0.00046,0.06011,0.00127,0.00012\n495,0.00264,0.45871,0.00002,0.00006,0.00008,0.00005,0.53010,0.00806,0.00028\n496,0.33004,0.00007,0.00014,0.66936,0.00011,0.00015,0.00010,0.00000,0.00003\n497,0.06085,0.00261,0.00192,0.03115,0.29727,0.28794,0.31825,0.00000,0.00000\n498,0.00014,0.34764,0.00002,0.00046,0.00055,0.00041,0.64928,0.00149,0.00001\n499,0.00045,0.69089,0.00001,0.21345,0.00007,0.00002,0.09504,0.00007,0.00000\n500,0.01359,0.02795,0.00514,0.01017,0.01086,0.01114,0.91179,0.00486,0.00450\n501,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n502,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n503,0.00001,0.15366,0.00001,0.00004,0.00027,0.00049,0.84495,0.00057,0.00000\n504,0.01933,0.07373,0.00711,0.19920,0.30265,0.01959,0.37820,0.00018,0.00000\n505,0.00018,0.51787,0.00005,0.00035,0.00341,0.00018,0.47327,0.00465,0.00004\n506,0.64299,0.00211,0.00355,0.14394,0.15664,0.05037,0.00029,0.00009,0.00002\n507,0.00003,0.10670,0.00159,0.00195,0.00039,0.51885,0.36961,0.00088,0.00000\n508,0.14446,0.01069,0.00004,0.83231,0.00002,0.00020,0.01177,0.00046,0.00005\n509,0.65783,0.00021,0.00000,0.02400,0.00000,0.00000,0.31796,0.00000,0.00000\n510,0.46803,0.15920,0.01403,0.07751,0.02564,0.02160,0.17347,0.02681,0.03372\n511,0.54247,0.25837,0.00002,0.00617,0.00100,0.00431,0.17603,0.01055,0.00108\n512,0.00003,0.80423,0.00000,0.00003,0.00014,0.00008,0.19526,0.00023,0.00000\n513,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n514,0.48985,0.00689,0.00048,0.47524,0.00043,0.00009,0.02673,0.00015,0.00013\n515,0.05831,0.01502,0.00010,0.91889,0.00004,0.00002,0.00759,0.00003,0.00000\n516,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n517,0.82217,0.00022,0.00007,0.00229,0.17457,0.00066,0.00002,0.00000,0.00000\n518,0.52351,0.33114,0.00002,0.06900,0.00037,0.00013,0.02699,0.04830,0.00054\n519,0.15011,0.45075,0.00019,0.02736,0.00563,0.01212,0.32911,0.02456,0.00018\n520,0.50193,0.00002,0.00002,0.47300,0.00000,0.00001,0.02502,0.00000,0.00000\n521,0.08442,0.49465,0.01806,0.06793,0.07278,0.02808,0.21293,0.01131,0.00984\n522,0.25224,0.17488,0.01696,0.06108,0.02083,0.03022,0.41694,0.01448,0.01238\n523,0.00004,0.42645,0.00002,0.00030,0.00025,0.00011,0.57203,0.00080,0.00000\n524,0.00288,0.04859,0.00009,0.55081,0.00002,0.00002,0.39751,0.00008,0.00000\n525,0.00464,0.01431,0.00010,0.60108,0.00001,0.00007,0.37979,0.00000,0.00000\n526,0.01623,0.01312,0.00369,0.02876,0.00514,0.00476,0.01771,0.00191,0.90868\n527,0.89354,0.00006,0.00004,0.10609,0.00016,0.00006,0.00004,0.00000,0.00001\n528,0.00003,0.12192,0.00009,0.00010,0.00159,0.00014,0.87502,0.00111,0.00000\n529,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n530,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n531,0.91081,0.00291,0.00014,0.07986,0.00014,0.00058,0.00081,0.00466,0.00009\n532,0.04902,0.20156,0.00042,0.31892,0.00031,0.00159,0.42642,0.00173,0.00002\n533,0.00099,0.15956,0.00010,0.00113,0.00028,0.00025,0.83468,0.00300,0.00001\n534,0.69570,0.03320,0.00001,0.00035,0.00001,0.00001,0.27072,0.00000,0.00000\n535,0.70903,0.00177,0.00029,0.00750,0.27754,0.00226,0.00145,0.00006,0.00010\n536,0.47869,0.38138,0.00001,0.04261,0.00046,0.00024,0.02797,0.06824,0.00041\n537,0.08507,0.04494,0.00059,0.00882,0.21956,0.00192,0.06097,0.03335,0.54478\n538,0.03155,0.23096,0.00160,0.10773,0.02122,0.01829,0.58814,0.00050,0.00001\n539,0.00195,0.20229,0.00452,0.07352,0.01491,0.11048,0.59176,0.00057,0.00000\n540,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n541,0.33049,0.59055,0.00000,0.01805,0.00018,0.00016,0.01591,0.04431,0.00035\n542,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n543,0.08042,0.07460,0.01397,0.08717,0.04613,0.02981,0.64021,0.01351,0.01418\n544,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n545,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n546,0.07626,0.51157,0.00456,0.06258,0.00465,0.01625,0.29953,0.02379,0.00081\n547,0.00005,0.35878,0.00159,0.00169,0.00070,0.22704,0.40908,0.00107,0.00000\n548,0.44289,0.05265,0.00000,0.00352,0.00182,0.00265,0.01799,0.07380,0.40467\n549,0.20338,0.00414,0.10463,0.67307,0.00690,0.00239,0.00548,0.00001,0.00000\n550,0.21217,0.18440,0.00017,0.32977,0.00241,0.00718,0.25919,0.00466,0.00005\n551,0.02793,0.00337,0.00000,0.00248,0.00066,0.01314,0.16155,0.26161,0.52926\n552,0.20013,0.77413,0.00000,0.00003,0.00000,0.00000,0.02570,0.00001,0.00000\n553,0.00085,0.39511,0.00155,0.00666,0.02978,0.10580,0.45448,0.00577,0.00000\n554,0.43068,0.25747,0.00013,0.04405,0.00096,0.00007,0.26315,0.00203,0.00146\n555,0.02198,0.02868,0.16502,0.03057,0.02620,0.01341,0.70023,0.00696,0.00695\n556,0.00006,0.75246,0.00001,0.00046,0.00040,0.00021,0.24594,0.00046,0.00000\n557,0.00017,0.02793,0.45105,0.00090,0.00772,0.00050,0.51173,0.00000,0.00000\n558,0.00120,0.74900,0.00015,0.18621,0.00406,0.00531,0.05376,0.00031,0.00000\n559,0.00077,0.01559,0.00000,0.00043,0.00060,0.94832,0.00238,0.03191,0.00000\n560,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n561,0.51114,0.01581,0.00939,0.05396,0.35232,0.01514,0.02693,0.00806,0.00725\n562,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n563,0.06387,0.02847,0.00064,0.23017,0.00807,0.00026,0.66830,0.00021,0.00000\n564,0.37500,0.00387,0.00038,0.61661,0.00138,0.00205,0.00068,0.00002,0.00001\n565,0.00002,0.05117,0.00009,0.00045,0.00005,0.00010,0.94805,0.00007,0.00000\n566,0.35014,0.06284,0.00010,0.04410,0.00180,0.02512,0.51568,0.00019,0.00003\n567,0.00699,0.87040,0.00000,0.00406,0.00055,0.00018,0.02029,0.01551,0.08202\n568,0.00072,0.42533,0.00001,0.02705,0.00004,0.00015,0.02441,0.52216,0.00013\n569,0.34199,0.00433,0.00000,0.00006,0.00001,0.00001,0.65355,0.00004,0.00001\n570,0.68190,0.27608,0.00002,0.02603,0.00003,0.00016,0.01560,0.00011,0.00007\n571,0.00015,0.01166,0.00000,0.00021,0.00003,0.00000,0.00176,0.05361,0.93258\n572,0.00001,0.07179,0.00005,0.00003,0.00012,0.00022,0.92770,0.00008,0.00000\n573,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n574,0.02090,0.01281,0.00005,0.18391,0.00002,0.00005,0.78218,0.00008,0.00000\n575,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n576,0.97512,0.00175,0.00004,0.00480,0.00005,0.00100,0.01715,0.00007,0.00002\n577,0.03569,0.04560,0.00925,0.00524,0.62191,0.00028,0.28026,0.00150,0.00027\n578,0.00626,0.00245,0.00017,0.52314,0.00007,0.46376,0.00414,0.00001,0.00000\n579,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n580,0.32276,0.00148,0.00018,0.67437,0.00031,0.00022,0.00066,0.00001,0.00001\n581,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n582,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n583,0.79760,0.05376,0.00671,0.02318,0.01130,0.01462,0.05537,0.00907,0.02839\n584,0.12050,0.00725,0.00012,0.51026,0.00009,0.00005,0.36171,0.00002,0.00000\n585,0.00679,0.00239,0.00061,0.35359,0.32979,0.00031,0.11947,0.01753,0.16953\n586,0.62855,0.02308,0.01465,0.04615,0.04337,0.20125,0.02353,0.00937,0.01005\n587,0.04150,0.15102,0.01231,0.05001,0.02172,0.02717,0.67440,0.01219,0.00968\n588,0.00117,0.00832,0.00005,0.63538,0.00001,0.00001,0.35505,0.00001,0.00000\n589,0.00000,0.02829,0.00002,0.00001,0.00003,0.00001,0.97158,0.00006,0.00000\n590,0.00026,0.36061,0.03691,0.00059,0.02616,0.00230,0.56774,0.00541,0.00002\n591,0.61984,0.12841,0.01306,0.09516,0.01809,0.02305,0.06739,0.02115,0.01384\n592,0.18178,0.00940,0.00007,0.42732,0.00003,0.00004,0.38107,0.00018,0.00010\n593,0.34978,0.01806,0.00140,0.61416,0.00154,0.00037,0.01416,0.00049,0.00004\n594,0.00001,0.07328,0.00001,0.00002,0.00004,0.00001,0.92633,0.00030,0.00000\n595,0.00034,0.30854,0.00041,0.00696,0.00045,0.00025,0.68197,0.00108,0.00000\n596,0.68455,0.29928,0.00000,0.00064,0.00005,0.00002,0.01542,0.00003,0.00001\n597,0.13886,0.02190,0.00827,0.76655,0.01575,0.01368,0.02218,0.00622,0.00659\n598,0.01134,0.23148,0.00001,0.29125,0.00008,0.00002,0.46526,0.00056,0.00000\n599,0.00246,0.01037,0.00216,0.90269,0.00749,0.00029,0.07453,0.00001,0.00000\n600,0.00021,0.60201,0.00001,0.00318,0.00037,0.00005,0.39308,0.00109,0.00000\n601,0.17180,0.44314,0.00022,0.02768,0.00312,0.00502,0.32139,0.02742,0.00021\n602,0.47582,0.36740,0.00002,0.05066,0.00040,0.00016,0.04618,0.05889,0.00048\n603,0.64360,0.00137,0.00125,0.07211,0.14288,0.13871,0.00007,0.00000,0.00001\n604,0.00098,0.54581,0.00001,0.00222,0.00045,0.00020,0.44881,0.00152,0.00000\n605,0.00004,0.96367,0.00001,0.00015,0.00007,0.01829,0.01762,0.00015,0.00000\n606,0.00140,0.00015,0.00003,0.93613,0.00000,0.00000,0.06229,0.00000,0.00000\n607,0.26506,0.00044,0.00008,0.67795,0.00000,0.00010,0.05634,0.00003,0.00000\n608,0.00002,0.59862,0.00000,0.00003,0.00003,0.00002,0.40120,0.00008,0.00000\n609,0.01223,0.00726,0.00001,0.39269,0.00003,0.00000,0.00064,0.03977,0.54737\n610,0.21865,0.00923,0.00022,0.00019,0.00002,0.00069,0.77098,0.00001,0.00001\n611,0.08931,0.16009,0.00201,0.08016,0.17710,0.00231,0.44800,0.03244,0.00859\n612,0.00002,0.07372,0.00001,0.00002,0.00009,0.00003,0.92458,0.00149,0.00004\n613,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n614,0.03624,0.56216,0.01534,0.03109,0.03517,0.02534,0.27577,0.00954,0.00935\n615,0.05272,0.13198,0.01471,0.08259,0.03053,0.02071,0.64610,0.01124,0.00942\n616,0.00001,0.05459,0.36840,0.00016,0.00958,0.00097,0.56628,0.00001,0.00000\n617,0.00011,0.03298,0.33578,0.00081,0.01065,0.00142,0.61823,0.00002,0.00000\n618,0.24848,0.04196,0.00366,0.35843,0.32414,0.00253,0.02050,0.00023,0.00007\n619,0.00231,0.00364,0.00051,0.57840,0.00034,0.40531,0.00768,0.00181,0.00000\n620,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n621,0.31234,0.64490,0.00000,0.00004,0.00005,0.00001,0.04262,0.00004,0.00000\n622,0.00122,0.02600,0.28960,0.00143,0.11501,0.00066,0.56600,0.00008,0.00000\n623,0.01384,0.83701,0.00699,0.01170,0.01296,0.01016,0.09868,0.00417,0.00449\n624,0.01674,0.00117,0.01541,0.25953,0.08897,0.01024,0.60794,0.00000,0.00000\n625,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n626,0.47308,0.22013,0.01606,0.12113,0.10861,0.00390,0.02119,0.03550,0.00039\n627,0.59988,0.01426,0.00000,0.00020,0.00002,0.00005,0.38555,0.00002,0.00002\n628,0.48561,0.36911,0.00001,0.03534,0.00036,0.00021,0.02900,0.07966,0.00071\n629,0.04883,0.11260,0.01387,0.05522,0.02253,0.60147,0.12574,0.01048,0.00926\n630,0.00017,0.20323,0.00001,0.00006,0.00044,0.00007,0.78693,0.00906,0.00003\n631,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n632,0.33819,0.32544,0.00004,0.03238,0.00043,0.00806,0.24640,0.04878,0.00028\n633,0.65953,0.02843,0.00000,0.00283,0.00002,0.00007,0.30897,0.00012,0.00003\n634,0.11788,0.02107,0.01487,0.43139,0.30199,0.05983,0.03580,0.00837,0.00880\n635,0.01321,0.08604,0.00788,0.01416,0.00975,0.00910,0.84923,0.00554,0.00509\n636,0.50868,0.01469,0.01106,0.03641,0.37903,0.01794,0.01733,0.00741,0.00745\n637,0.56309,0.19891,0.00008,0.07163,0.00256,0.15810,0.00359,0.00202,0.00002\n638,0.00427,0.04706,0.00518,0.83731,0.00062,0.02151,0.08404,0.00001,0.00000\n639,0.01814,0.00460,0.00029,0.87039,0.00195,0.00040,0.10423,0.00000,0.00000\n640,0.30547,0.41683,0.00008,0.12568,0.00613,0.02368,0.12105,0.00094,0.00014\n641,0.04591,0.01163,0.00003,0.94096,0.00001,0.00004,0.00124,0.00018,0.00000\n642,0.28717,0.00010,0.00007,0.71230,0.00020,0.00005,0.00011,0.00000,0.00000\n643,0.54405,0.30791,0.00004,0.07389,0.00043,0.00014,0.02640,0.04651,0.00063\n644,0.02953,0.10894,0.00909,0.02957,0.01784,0.01963,0.75959,0.01265,0.01316\n645,0.03315,0.00909,0.01987,0.02069,0.82738,0.06964,0.00992,0.00526,0.00500\n646,0.18894,0.78529,0.00001,0.00012,0.00000,0.00002,0.00599,0.00311,0.01652\n647,0.66563,0.01901,0.00000,0.00020,0.00001,0.00001,0.31386,0.00052,0.00076\n648,0.73260,0.02359,0.00884,0.14899,0.02866,0.01497,0.02439,0.00837,0.00959\n649,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n650,0.02950,0.00573,0.00010,0.49427,0.00019,0.01892,0.45113,0.00016,0.00001\n651,0.06010,0.06418,0.00127,0.00847,0.62190,0.00038,0.24135,0.00207,0.00028\n652,0.09135,0.00275,0.00011,0.90533,0.00005,0.00002,0.00039,0.00000,0.00000\n653,0.65649,0.00173,0.00237,0.22600,0.00152,0.11169,0.00019,0.00001,0.00000\n654,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n655,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n656,0.26454,0.00413,0.00008,0.73094,0.00000,0.00005,0.00026,0.00000,0.00000\n657,0.02039,0.08891,0.01269,0.03563,0.01865,0.03130,0.77328,0.01036,0.00879\n658,0.36567,0.00407,0.00071,0.58563,0.00601,0.02806,0.00058,0.00571,0.00356\n659,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n660,0.51878,0.33871,0.00001,0.05544,0.00040,0.00018,0.02715,0.05868,0.00065\n661,0.00034,0.00666,0.00008,0.43784,0.00001,0.00001,0.55506,0.00000,0.00000\n662,0.00008,0.72784,0.00000,0.00132,0.00025,0.00007,0.26983,0.00061,0.00000\n663,0.80465,0.01788,0.00837,0.10613,0.01344,0.01306,0.02328,0.00578,0.00741\n664,0.00001,0.37255,0.00000,0.00001,0.00005,0.00003,0.62722,0.00013,0.00000\n665,0.99992,0.00000,0.00000,0.00008,0.00000,0.00000,0.00000,0.00000,0.00000\n666,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n667,0.00007,0.86376,0.00000,0.00010,0.00020,0.00007,0.13523,0.00057,0.00000\n668,0.50093,0.35293,0.00005,0.04820,0.00051,0.00051,0.02798,0.06821,0.00068\n669,0.00965,0.04070,0.01098,0.03863,0.08543,0.00013,0.81380,0.00046,0.00022\n670,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n671,0.46755,0.00112,0.00000,0.00333,0.00001,0.00002,0.52793,0.00002,0.00002\n672,0.83834,0.00138,0.00005,0.14041,0.00030,0.00013,0.00060,0.00042,0.01837\n673,0.00076,0.01325,0.00026,0.50159,0.00012,0.00002,0.48395,0.00005,0.00000\n674,0.01128,0.91702,0.00450,0.01514,0.01304,0.00586,0.02643,0.00337,0.00336\n675,0.00131,0.38260,0.00007,0.53916,0.00174,0.03018,0.04406,0.00088,0.00000\n676,0.01912,0.06997,0.00307,0.08453,0.02309,0.04779,0.75201,0.00042,0.00000\n677,0.17645,0.00007,0.00910,0.38931,0.41278,0.01218,0.00009,0.00000,0.00001\n678,0.03773,0.00047,0.00002,0.55106,0.00001,0.00001,0.41070,0.00000,0.00000\n679,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n680,0.58714,0.00106,0.00021,0.40353,0.00428,0.00304,0.00069,0.00001,0.00004\n681,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n682,0.30246,0.01099,0.00000,0.00029,0.00001,0.00036,0.68589,0.00000,0.00000\n683,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n684,0.06836,0.00016,0.00003,0.68213,0.00002,0.00001,0.24929,0.00000,0.00000\n685,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n686,0.00862,0.88488,0.00006,0.01496,0.00010,0.01935,0.06985,0.00216,0.00002\n687,0.69323,0.26623,0.00000,0.00195,0.00021,0.00019,0.03542,0.00270,0.00007\n688,0.54285,0.01121,0.00013,0.04010,0.00290,0.00161,0.40066,0.00051,0.00003\n689,0.67171,0.00154,0.00000,0.00009,0.00000,0.00001,0.32665,0.00000,0.00000\n690,0.25019,0.00634,0.00082,0.70824,0.00113,0.01984,0.01338,0.00004,0.00002\n691,0.08917,0.18683,0.01827,0.30873,0.03425,0.02904,0.29613,0.01861,0.01897\n692,0.17331,0.17822,0.01957,0.39106,0.04114,0.06888,0.10366,0.01204,0.01211\n693,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n694,0.56249,0.00039,0.00008,0.43652,0.00005,0.00002,0.00045,0.00000,0.00000\n695,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n696,0.51224,0.00125,0.00001,0.04892,0.00018,0.00448,0.43286,0.00003,0.00003\n697,0.45573,0.03229,0.00001,0.00022,0.00016,0.00035,0.51100,0.00023,0.00001\n698,0.00003,0.93220,0.00000,0.00006,0.00016,0.00009,0.06647,0.00099,0.00000\n699,0.08466,0.20463,0.01261,0.04530,0.02358,0.03591,0.57444,0.00925,0.00962\n700,0.00001,0.01424,0.00001,0.00001,0.00001,0.00001,0.98570,0.00001,0.00000\n701,0.01760,0.01486,0.00004,0.58571,0.00001,0.00002,0.38175,0.00001,0.00000\n702,0.05308,0.02346,0.00001,0.00172,0.00002,0.00000,0.92158,0.00006,0.00007\n703,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n704,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n705,0.36862,0.00057,0.00010,0.63023,0.00015,0.00005,0.00028,0.00000,0.00000\n706,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n707,0.02539,0.05958,0.01058,0.65393,0.14172,0.01152,0.09613,0.00112,0.00003\n708,0.30162,0.04946,0.02156,0.20392,0.07276,0.04202,0.27719,0.01668,0.01479\n709,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n710,0.20395,0.40620,0.00031,0.03555,0.00391,0.00546,0.31696,0.02740,0.00026\n711,0.57668,0.02375,0.00358,0.10270,0.02582,0.26232,0.00510,0.00004,0.00001\n712,0.00596,0.00990,0.00000,0.00161,0.00142,0.00003,0.01175,0.14032,0.82901\n713,0.04353,0.17698,0.01386,0.04958,0.02205,0.03740,0.63650,0.00970,0.01040\n714,0.02355,0.06832,0.00192,0.00919,0.05495,0.00017,0.83976,0.00201,0.00013\n715,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n716,0.02038,0.01722,0.01333,0.25296,0.11628,0.09686,0.47516,0.00779,0.00001\n717,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n718,0.05142,0.08993,0.01488,0.02798,0.03357,0.02090,0.74689,0.00713,0.00730\n719,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n720,0.01690,0.05185,0.00008,0.04013,0.00007,0.00134,0.88862,0.00080,0.00021\n721,0.29619,0.11611,0.00002,0.58622,0.00003,0.00012,0.00059,0.00058,0.00014\n722,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n723,0.02198,0.02868,0.16502,0.03057,0.02620,0.01341,0.70023,0.00696,0.00695\n724,0.31210,0.04534,0.00005,0.63336,0.00002,0.00012,0.00868,0.00032,0.00001\n725,0.71754,0.02893,0.00000,0.00027,0.00000,0.00006,0.22475,0.00519,0.02326\n726,0.03141,0.02626,0.00015,0.77966,0.00035,0.00004,0.15847,0.00156,0.00210\n727,0.08277,0.13461,0.00044,0.03085,0.00048,0.00240,0.74786,0.00057,0.00002\n728,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n729,0.49842,0.00944,0.00093,0.48555,0.00265,0.00099,0.00178,0.00017,0.00007\n730,0.48638,0.19299,0.01653,0.15294,0.07033,0.00415,0.04160,0.03456,0.00053\n731,0.02015,0.02207,0.00013,0.45179,0.00003,0.00002,0.50580,0.00001,0.00000\n732,0.00019,0.02252,0.00005,0.00106,0.00004,0.26944,0.70661,0.00009,0.00000\n733,0.03532,0.01259,0.00065,0.64731,0.00004,0.00165,0.30234,0.00007,0.00003\n734,0.00046,0.01404,0.02712,0.00028,0.03994,0.00005,0.91777,0.00032,0.00002\n735,0.02568,0.15250,0.00961,0.02799,0.02520,0.54496,0.19729,0.00854,0.00823\n736,0.00004,0.35087,0.00001,0.00027,0.00034,0.00040,0.64278,0.00511,0.00018\n737,0.16816,0.40560,0.01701,0.04126,0.04258,0.04039,0.24107,0.03011,0.01382\n738,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n739,0.02598,0.18347,0.00027,0.28981,0.00028,0.00490,0.49507,0.00022,0.00000\n740,0.00002,0.22208,0.00006,0.00005,0.00043,0.00012,0.77655,0.00069,0.00000\n741,0.00227,0.28965,0.00011,0.01541,0.00083,0.00067,0.69017,0.00089,0.00000\n742,0.00094,0.83837,0.00001,0.00399,0.00029,0.00007,0.15574,0.00059,0.00000\n743,0.05297,0.00033,0.00003,0.94660,0.00001,0.00000,0.00006,0.00000,0.00000\n744,0.00002,0.23601,0.00005,0.00033,0.00025,0.00009,0.76282,0.00043,0.00000\n745,0.01039,0.00700,0.53048,0.02270,0.04173,0.00122,0.00789,0.02449,0.35410\n746,0.18163,0.05820,0.00014,0.14396,0.00038,0.00058,0.61439,0.00071,0.00001\n747,0.46872,0.00665,0.00009,0.49994,0.00025,0.01087,0.01326,0.00021,0.00002\n748,0.28651,0.01007,0.00000,0.00878,0.00001,0.00002,0.68773,0.00019,0.00669\n749,0.65427,0.02333,0.00006,0.26503,0.00032,0.00016,0.05664,0.00010,0.00009\n750,0.00318,0.03929,0.00002,0.71400,0.00001,0.00000,0.24347,0.00003,0.00000\n751,0.00636,0.00266,0.00001,0.41690,0.00001,0.00001,0.57404,0.00001,0.00000\n752,0.43872,0.01245,0.00882,0.03596,0.44518,0.03135,0.01405,0.00687,0.00659\n753,0.00043,0.78354,0.00001,0.01653,0.00008,0.00002,0.19889,0.00050,0.00000\n754,0.24410,0.02020,0.04475,0.67815,0.00463,0.00660,0.00131,0.00025,0.00001\n755,0.91398,0.00059,0.00230,0.02440,0.00015,0.00008,0.05847,0.00000,0.00003\n756,0.00089,0.25848,0.00030,0.00108,0.00537,0.00073,0.73137,0.00175,0.00003\n757,0.08189,0.00157,0.88324,0.02406,0.00377,0.00027,0.00520,0.00000,0.00000\n758,0.18375,0.03730,0.02215,0.65804,0.02109,0.02588,0.03203,0.00971,0.01005\n759,0.01809,0.00755,0.00421,0.01322,0.01487,0.93100,0.00582,0.00265,0.00259\n760,0.00800,0.91193,0.00001,0.00485,0.02672,0.00015,0.04785,0.00048,0.00001\n761,0.02627,0.02410,0.00062,0.00150,0.00132,0.00007,0.94603,0.00008,0.00001\n762,0.07582,0.02777,0.02078,0.27962,0.18151,0.02732,0.38710,0.00008,0.00001\n763,0.54246,0.00782,0.00060,0.43731,0.00059,0.00013,0.01084,0.00015,0.00010\n764,0.00601,0.65556,0.00007,0.00294,0.00056,0.00006,0.33236,0.00235,0.00009\n765,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n766,0.00582,0.48590,0.00124,0.00468,0.33504,0.01088,0.14943,0.00699,0.00002\n767,0.07680,0.22456,0.01156,0.08720,0.02472,0.02854,0.52533,0.01025,0.01104\n768,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n769,0.00055,0.38871,0.00062,0.00183,0.00826,0.00875,0.58802,0.00325,0.00001\n770,0.33855,0.01450,0.00021,0.61671,0.00253,0.00444,0.02269,0.00027,0.00010\n771,0.02232,0.61869,0.00999,0.01821,0.01498,0.02791,0.27573,0.00670,0.00547\n772,0.00002,0.53981,0.00006,0.00002,0.00031,0.00038,0.45737,0.00203,0.00000\n773,0.78610,0.01631,0.00750,0.13443,0.01623,0.01203,0.01660,0.00538,0.00542\n774,0.43820,0.51547,0.00001,0.02565,0.00001,0.00929,0.01133,0.00004,0.00000\n775,0.43872,0.01245,0.00882,0.03596,0.44518,0.03135,0.01405,0.00687,0.00659\n776,0.43759,0.39845,0.00002,0.06900,0.00046,0.00027,0.03321,0.06013,0.00088\n777,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n778,0.25783,0.20584,0.00005,0.03351,0.00055,0.00018,0.47173,0.03013,0.00018\n779,0.68264,0.00378,0.00003,0.00015,0.00000,0.00003,0.31336,0.00000,0.00001\n780,0.01713,0.25048,0.00019,0.00086,0.00301,0.00376,0.72137,0.00317,0.00003\n781,0.00001,0.04988,0.00002,0.00002,0.00013,0.00001,0.94989,0.00004,0.00000\n782,0.95196,0.00209,0.00000,0.04564,0.00003,0.00005,0.00016,0.00003,0.00004\n783,0.00362,0.04933,0.01000,0.00898,0.00323,0.00063,0.92414,0.00007,0.00000\n784,0.50868,0.01469,0.01106,0.03641,0.37903,0.01794,0.01733,0.00741,0.00745\n785,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n786,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n787,0.00234,0.02492,0.00007,0.59425,0.00004,0.00004,0.37827,0.00007,0.00000\n788,0.02996,0.27247,0.01200,0.03340,0.02613,0.04766,0.55777,0.01080,0.00981\n789,0.61748,0.05272,0.01837,0.13241,0.04674,0.04668,0.05754,0.01657,0.01149\n790,0.04907,0.24355,0.00037,0.43517,0.00020,0.01089,0.26009,0.00066,0.00001\n791,0.44560,0.01212,0.00000,0.01099,0.00001,0.00054,0.53037,0.00006,0.00031\n792,0.04472,0.17794,0.01446,0.04967,0.02545,0.03075,0.63727,0.01079,0.00895\n793,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n794,0.00838,0.03333,0.00883,0.02325,0.24313,0.01475,0.66812,0.00018,0.00003\n795,0.05483,0.57673,0.01578,0.05141,0.02913,0.03235,0.21625,0.01170,0.01182\n796,0.06871,0.00005,0.00005,0.93112,0.00003,0.00002,0.00002,0.00000,0.00000\n797,0.48997,0.35737,0.00001,0.05235,0.00044,0.00026,0.02753,0.07163,0.00044\n798,0.50165,0.35649,0.00001,0.03689,0.00042,0.00022,0.02865,0.07489,0.00078\n799,0.80465,0.01788,0.00837,0.10613,0.01344,0.01306,0.02328,0.00578,0.00741\n800,0.00723,0.41749,0.00037,0.00199,0.01351,0.01640,0.53487,0.00804,0.00010\n801,0.45806,0.39494,0.00001,0.03976,0.00052,0.00037,0.02686,0.07902,0.00046\n802,0.25286,0.11162,0.00003,0.00066,0.00059,0.62151,0.01202,0.00071,0.00000\n803,0.03558,0.10743,0.01294,0.06063,0.02967,0.01910,0.71672,0.00893,0.00900\n804,0.03023,0.00025,0.17199,0.78673,0.00649,0.00273,0.00158,0.00000,0.00000\n805,0.00021,0.40981,0.00002,0.00007,0.00012,0.00119,0.58791,0.00067,0.00000\n806,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n807,0.94309,0.00390,0.00030,0.04028,0.00035,0.00722,0.00479,0.00002,0.00005\n808,0.92457,0.03145,0.00018,0.00215,0.00005,0.00003,0.04153,0.00001,0.00003\n809,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n810,0.48953,0.00045,0.00010,0.50895,0.00058,0.00016,0.00022,0.00000,0.00001\n811,0.53059,0.00053,0.00055,0.00694,0.45946,0.00118,0.00070,0.00001,0.00004\n812,0.00010,0.50717,0.00014,0.00019,0.00123,0.00014,0.48899,0.00204,0.00000\n813,0.00017,0.86421,0.00000,0.00014,0.00025,0.00007,0.13327,0.00188,0.00001\n814,0.18337,0.01167,0.00017,0.51993,0.00069,0.00056,0.28357,0.00003,0.00001\n815,0.24593,0.73590,0.00000,0.00402,0.00007,0.00005,0.01384,0.00019,0.00000\n816,0.32268,0.29812,0.01270,0.05900,0.02828,0.03575,0.21962,0.01319,0.01066\n817,0.13026,0.02808,0.01387,0.72872,0.02611,0.01856,0.03794,0.00841,0.00805\n818,0.68156,0.00063,0.00030,0.31468,0.00117,0.00015,0.00036,0.00003,0.00112\n819,0.00448,0.39667,0.00024,0.00503,0.00230,0.01248,0.57324,0.00526,0.00030\n820,0.60956,0.00043,0.00010,0.00798,0.11508,0.26670,0.00014,0.00001,0.00000\n821,0.01900,0.03243,0.63036,0.02269,0.06675,0.00993,0.20605,0.00622,0.00657\n822,0.00097,0.05409,0.00005,0.00096,0.00096,0.00006,0.94244,0.00046,0.00001\n823,0.01128,0.91702,0.00450,0.01514,0.01304,0.00586,0.02643,0.00337,0.00336\n824,0.49872,0.01053,0.00005,0.46905,0.00011,0.00040,0.02015,0.00097,0.00002\n825,0.65999,0.02201,0.00033,0.01835,0.00142,0.00007,0.29650,0.00124,0.00009\n826,0.00088,0.14897,0.07937,0.00223,0.14411,0.00230,0.62115,0.00099,0.00000\n827,0.00637,0.00002,0.06068,0.93153,0.00051,0.00058,0.00031,0.00000,0.00000\n828,0.00047,0.03718,0.00007,0.00140,0.00037,0.92129,0.03882,0.00040,0.00000\n829,0.52272,0.00927,0.00003,0.00020,0.00006,0.00002,0.46770,0.00000,0.00000\n830,0.63488,0.00235,0.00001,0.09001,0.00000,0.00001,0.27272,0.00000,0.00002\n831,0.04006,0.00024,0.03157,0.06386,0.82058,0.04338,0.00026,0.00000,0.00005\n832,0.06828,0.25526,0.01635,0.07233,0.05329,0.02822,0.48605,0.01131,0.00890\n833,0.73916,0.00055,0.00039,0.25654,0.00282,0.00021,0.00032,0.00000,0.00001\n834,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n835,0.00012,0.00446,0.00000,0.00008,0.00009,0.00001,0.50913,0.03274,0.45337\n836,0.65650,0.00856,0.00006,0.33334,0.00028,0.00011,0.00079,0.00024,0.00012\n837,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n838,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n839,0.00750,0.03529,0.00184,0.93769,0.00436,0.00454,0.00872,0.00006,0.00000\n840,0.00007,0.06321,0.00000,0.00007,0.00001,0.00003,0.93660,0.00001,0.00000\n841,0.00464,0.77677,0.00001,0.00673,0.00029,0.00006,0.21111,0.00039,0.00000\n842,0.37433,0.00855,0.00001,0.01256,0.00001,0.56593,0.03853,0.00006,0.00002\n843,0.76354,0.04835,0.00000,0.02448,0.00003,0.00003,0.16346,0.00010,0.00001\n844,0.16487,0.31209,0.00027,0.16381,0.00194,0.00961,0.34202,0.00411,0.00128\n845,0.05084,0.17625,0.01608,0.05192,0.02369,0.02413,0.63649,0.01185,0.00875\n846,0.00334,0.30657,0.00019,0.02951,0.00225,0.01944,0.63608,0.00261,0.00001\n847,0.00003,0.43298,0.00003,0.00049,0.00035,0.00008,0.56577,0.00027,0.00000\n848,0.32680,0.19097,0.00012,0.04464,0.00165,0.06027,0.37337,0.00218,0.00001\n849,0.00001,0.15063,0.00009,0.00007,0.00024,0.00009,0.84866,0.00021,0.00000\n850,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n851,0.22643,0.02052,0.01101,0.12731,0.01887,0.55467,0.02650,0.00740,0.00729\n852,0.01625,0.21690,0.00246,0.09054,0.03759,0.12159,0.51456,0.00011,0.00000\n853,0.00168,0.15206,0.01132,0.00033,0.04377,0.00073,0.77972,0.01036,0.00003\n854,0.03291,0.00997,0.00013,0.61600,0.00014,0.00003,0.34078,0.00004,0.00000\n855,0.55049,0.02634,0.00981,0.33152,0.01632,0.01856,0.02907,0.00769,0.01020\n856,0.60437,0.34656,0.00787,0.00700,0.00253,0.01932,0.01192,0.00030,0.00013\n857,0.00602,0.13635,0.00066,0.70762,0.00089,0.00086,0.14757,0.00003,0.00000\n858,0.04689,0.04149,0.01230,0.59126,0.22266,0.02354,0.04655,0.00745,0.00786\n859,0.00002,0.06225,0.36468,0.00047,0.00722,0.00483,0.56052,0.00001,0.00000\n860,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n861,0.29410,0.31834,0.00046,0.05519,0.00223,0.00136,0.27929,0.04842,0.00060\n862,0.51273,0.34196,0.00001,0.04835,0.00053,0.00035,0.02741,0.06810,0.00056\n863,0.30912,0.01133,0.00145,0.02224,0.00023,0.00035,0.65486,0.00019,0.00023\n864,0.00024,0.01616,0.04123,0.00014,0.03506,0.00010,0.90655,0.00052,0.00000\n865,0.42796,0.02423,0.00018,0.54591,0.00087,0.00029,0.00053,0.00001,0.00002\n866,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n867,0.01216,0.03061,0.00738,0.06299,0.04780,0.00026,0.83845,0.00029,0.00006\n868,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n869,0.00669,0.05507,0.00002,0.00460,0.00008,0.00003,0.93320,0.00031,0.00000\n870,0.00016,0.53789,0.00004,0.00483,0.00061,0.00021,0.45533,0.00093,0.00000\n871,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n872,0.06629,0.65826,0.00093,0.01622,0.02739,0.03365,0.18258,0.01464,0.00004\n873,0.00006,0.61452,0.00000,0.00015,0.00027,0.00005,0.38399,0.00096,0.00000\n874,0.33135,0.45319,0.00008,0.17233,0.00095,0.00291,0.03836,0.00077,0.00007\n875,0.02725,0.06560,0.01245,0.02144,0.02250,0.01390,0.82112,0.00732,0.00842\n876,0.01102,0.41286,0.00014,0.54675,0.00002,0.00025,0.02895,0.00001,0.00000\n877,0.60682,0.06054,0.03074,0.16244,0.02330,0.03913,0.05162,0.01322,0.01219\n878,0.00067,0.54361,0.00030,0.00147,0.00671,0.00858,0.43592,0.00273,0.00001\n879,0.01909,0.00013,0.44437,0.52962,0.00229,0.00380,0.00070,0.00000,0.00000\n880,0.73754,0.03518,0.00024,0.11436,0.00405,0.10377,0.00455,0.00029,0.00002\n881,0.00030,0.70842,0.00000,0.00064,0.00031,0.00005,0.28908,0.00119,0.00001\n882,0.07465,0.25462,0.01748,0.05885,0.04539,0.07303,0.44648,0.01612,0.01337\n883,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n884,0.02358,0.00261,0.00547,0.03305,0.57400,0.00233,0.35879,0.00016,0.00001\n885,0.04158,0.23722,0.01280,0.04548,0.02412,0.03184,0.56217,0.03495,0.00984\n886,0.81439,0.00151,0.00034,0.00481,0.17500,0.00091,0.00291,0.00003,0.00010\n887,0.50868,0.01469,0.01106,0.03641,0.37903,0.01794,0.01733,0.00741,0.00745\n888,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n889,0.48790,0.35896,0.00001,0.04168,0.00052,0.00029,0.03274,0.07738,0.00051\n890,0.00294,0.35851,0.00058,0.00208,0.11853,0.00125,0.50331,0.01274,0.00006\n891,0.00022,0.35697,0.00001,0.00018,0.00038,0.00031,0.64094,0.00098,0.00001\n892,0.00001,0.09342,0.00001,0.00002,0.00006,0.00413,0.90125,0.00109,0.00001\n893,0.00006,0.25950,0.00005,0.00025,0.00095,0.00138,0.73673,0.00107,0.00001\n894,0.00002,0.02916,0.34073,0.00019,0.00819,0.00401,0.61769,0.00001,0.00000\n895,0.71572,0.05480,0.00787,0.09684,0.01359,0.01447,0.08122,0.00646,0.00903\n896,0.07026,0.11558,0.01461,0.05057,0.02966,0.03448,0.66175,0.01042,0.01267\n897,0.88090,0.05916,0.00000,0.02242,0.00129,0.00011,0.03474,0.00127,0.00011\n898,0.03492,0.15451,0.01069,0.02875,0.01602,0.01533,0.72252,0.00918,0.00808\n899,0.00010,0.89414,0.00000,0.00002,0.00004,0.00002,0.10402,0.00165,0.00001\n900,0.99841,0.00018,0.00000,0.00012,0.00012,0.00099,0.00003,0.00014,0.00001\n901,0.04103,0.22826,0.03315,0.06540,0.08096,0.06204,0.46037,0.01586,0.01292\n902,0.48288,0.25674,0.00241,0.09973,0.06196,0.00757,0.03638,0.05178,0.00055\n903,0.43872,0.01245,0.00882,0.03596,0.44518,0.03135,0.01405,0.00687,0.00659\n904,0.02120,0.00054,0.00432,0.24565,0.07933,0.00458,0.64437,0.00000,0.00000\n905,0.03994,0.22096,0.03233,0.02066,0.49764,0.00598,0.17661,0.00584,0.00004\n906,0.00010,0.00132,0.00000,0.00015,0.00008,0.00000,0.00132,0.06636,0.93067\n907,0.07117,0.17740,0.02309,0.05115,0.04264,0.07227,0.54201,0.01011,0.01016\n908,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n909,0.00078,0.53520,0.01089,0.00165,0.00320,0.02234,0.41384,0.01203,0.00007\n910,0.00314,0.03009,0.00007,0.50273,0.00004,0.00421,0.45969,0.00003,0.00000\n911,0.04779,0.01466,0.00022,0.36248,0.02585,0.51328,0.03415,0.00157,0.00000\n912,0.76160,0.01814,0.01017,0.08778,0.03875,0.01806,0.05011,0.00788,0.00751\n913,0.01875,0.18505,0.00053,0.59264,0.00064,0.00067,0.20163,0.00009,0.00000\n914,0.00003,0.65069,0.00000,0.00012,0.00028,0.00004,0.34838,0.00046,0.00000\n915,0.00009,0.00064,0.00006,0.00038,0.00001,0.21577,0.78305,0.00000,0.00000\n916,0.11035,0.06197,0.01284,0.05109,0.01952,0.02097,0.70369,0.01076,0.00881\n917,0.00002,0.24837,0.00003,0.00014,0.00043,0.00019,0.74903,0.00179,0.00000\n918,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n919,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n920,0.26219,0.06506,0.02373,0.41595,0.04261,0.03785,0.12753,0.01269,0.01238\n921,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000\n922,0.70376,0.00854,0.00000,0.00005,0.00001,0.00000,0.28760,0.00002,0.00002\n923,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n924,0.00022,0.45662,0.00427,0.00045,0.00159,0.00918,0.52549,0.00217,0.00001\n925,0.65118,0.02774,0.00005,0.32040,0.00003,0.00017,0.00037,0.00002,0.00004\n926,0.00198,0.39416,0.00068,0.49047,0.00059,0.01519,0.09677,0.00017,0.00000\n927,0.00032,0.14437,0.00026,0.01602,0.00180,0.52617,0.30992,0.00114,0.00000\n928,0.04360,0.12955,0.01245,0.04152,0.04367,0.02855,0.66756,0.02163,0.01147\n929,0.46529,0.00884,0.00002,0.01600,0.00026,0.00003,0.50941,0.00014,0.00001\n930,0.04814,0.36194,0.01410,0.04451,0.04127,0.04136,0.42762,0.01124,0.00982\n931,0.02545,0.32856,0.00014,0.00092,0.00038,0.29481,0.34802,0.00173,0.00000\n932,0.22074,0.03558,0.00007,0.73880,0.00008,0.00011,0.00289,0.00146,0.00026\n933,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n934,0.78527,0.00166,0.00005,0.21187,0.00027,0.00006,0.00071,0.00003,0.00008\n935,0.48013,0.01091,0.00049,0.47996,0.00055,0.00012,0.02736,0.00037,0.00012\n936,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n937,0.00253,0.00257,0.00004,0.59635,0.00002,0.00137,0.39710,0.00002,0.00000\n938,0.66239,0.11427,0.00001,0.00026,0.00009,0.00006,0.21964,0.00318,0.00011\n939,0.00008,0.38401,0.00067,0.00162,0.00423,0.00072,0.60800,0.00067,0.00000\n940,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n941,0.00002,0.20885,0.00001,0.00003,0.00019,0.00003,0.78958,0.00129,0.00000\n942,0.00007,0.86499,0.00001,0.00047,0.00042,0.00015,0.13359,0.00030,0.00000\n943,0.14871,0.00031,0.01925,0.36296,0.43512,0.03354,0.00011,0.00000,0.00001\n944,0.06188,0.17319,0.01654,0.13729,0.03584,0.02776,0.52488,0.01143,0.01119\n945,0.00086,0.05588,0.00001,0.00016,0.00004,0.00005,0.94281,0.00019,0.00000\n946,0.00417,0.19796,0.02483,0.07801,0.05873,0.01398,0.61385,0.00830,0.00018\n947,0.00014,0.28027,0.00009,0.00008,0.00596,0.00036,0.58786,0.12400,0.00124\n948,0.05273,0.14888,0.01612,0.14733,0.04337,0.02581,0.54075,0.01344,0.01157\n949,0.20852,0.57038,0.00004,0.20432,0.00018,0.00018,0.01626,0.00011,0.00001\n950,0.24922,0.02643,0.00000,0.00005,0.00001,0.00001,0.72426,0.00002,0.00000\n951,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n952,0.15733,0.03570,0.01402,0.58844,0.02032,0.01947,0.14636,0.00901,0.00935\n953,0.10947,0.17092,0.01310,0.03469,0.02930,0.02936,0.58587,0.01519,0.01210\n954,0.00731,0.01930,0.00002,0.26583,0.00001,0.00001,0.70752,0.00000,0.00000\n955,0.41127,0.00341,0.00055,0.57191,0.00243,0.00980,0.00060,0.00002,0.00001\n956,0.06823,0.09206,0.40609,0.07884,0.08142,0.01853,0.23768,0.00866,0.00849\n957,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n958,0.00029,0.20588,0.00750,0.00119,0.02786,0.00349,0.75197,0.00181,0.00001\n959,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n960,0.62724,0.00476,0.00001,0.02470,0.00004,0.00017,0.34233,0.00039,0.00036\n961,0.14700,0.00785,0.38963,0.05596,0.00704,0.00035,0.39216,0.00001,0.00000\n962,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n963,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n964,0.61836,0.07468,0.00294,0.03135,0.00411,0.00086,0.23726,0.02984,0.00060\n965,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000\n966,0.00090,0.28289,0.00040,0.00043,0.00262,0.00231,0.70837,0.00202,0.00006\n967,0.30436,0.00304,0.00015,0.69075,0.00046,0.00023,0.00097,0.00003,0.00001\n968,0.00119,0.70169,0.00002,0.00133,0.00096,0.00014,0.27888,0.01574,0.00005\n969,0.94496,0.00221,0.00000,0.04833,0.00000,0.00001,0.00421,0.00014,0.00014\n970,0.11244,0.21410,0.00001,0.00005,0.00014,0.00005,0.67242,0.00079,0.00000\n971,0.82590,0.02832,0.00726,0.06955,0.01551,0.01128,0.02887,0.00759,0.00572\n972,0.68853,0.03678,0.01273,0.16300,0.02002,0.01811,0.03964,0.00984,0.01135\n973,0.46955,0.39852,0.00001,0.03807,0.00040,0.00026,0.02686,0.06587,0.00047\n974,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n975,0.00066,0.00040,0.00001,0.33245,0.00000,0.00000,0.66648,0.00000,0.00000\n976,0.63008,0.00441,0.00000,0.02012,0.00002,0.00001,0.34449,0.00057,0.00030\n977,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n978,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000\n979,0.00486,0.28569,0.00016,0.02157,0.00152,0.01837,0.66572,0.00211,0.00000\n980,0.00674,0.95974,0.00000,0.00470,0.00006,0.00005,0.02185,0.00678,0.00008\n981,0.00008,0.00516,0.00007,0.00028,0.00021,0.42714,0.56689,0.00017,0.00000\n982,0.56756,0.05815,0.00002,0.37369,0.00000,0.00002,0.00048,0.00001,0.00007\n983,0.00055,0.34936,0.00000,0.00014,0.00005,0.00022,0.11107,0.53791,0.00070\n984,0.02014,0.63020,0.00942,0.02367,0.02252,0.02904,0.24806,0.00875,0.00820\n985,0.97137,0.01867,0.00000,0.00129,0.00003,0.00739,0.00046,0.00069,0.00010\n986,0.39910,0.13265,0.00149,0.05070,0.03625,0.00073,0.37437,0.00463,0.00007\n'), index_col='ID')
df_subs.append(df_sub1)
df_subs.append(df_sub2)

In [ ]:
seed(999)
np.random.seed(999)

NGEN = 200 # nb of generations
MU = 500 # size of pop
LAMBDA = 500 # size of children
CXPB = 0.7 # prop of crossover
MUTPB = 0.3 # prop of mutation

#### ALGO GEN
toolbox = base.Toolbox()

score = 3.17899 # change to your private leaderboard score
perc_to_pick = 0.24
mut_count_max = 20

helper = Helper(df_subs, score, perc_to_pick, mut_count_max)
toolbox.register("individual", Ind, helper)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", helper.evalInd)
toolbox.register("mate", helper.cxInd)
toolbox.register("mutate", helper.mutInd)
toolbox.register("select", tools.selNSGA2)

#### exec
pop = toolbox.population(n=MU)
hof = tools.ParetoFront()
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats, halloffame=hof)

#### print best team
print('HOF')
hof[0].prettyPrint()
print(helper.evalInd(hof[0]))